# モジュールインポート

In [2]:
import os
import time
import json
import math
import uuid
import shutil
import math
import logging
from dataclasses import dataclass, asdict, field
from datetime import datetime, timedelta, timezone
from typing import Optional, Dict, Any, List, Tuple
import pytz

import pandas as pd

# 基本設定

## タイムゾーン設定

In [3]:
TZ_UTC = timezone.utc
try:
    TZ_JST = pytz.timezone("Asia/Tokyo")
except Exception:
    # pytzがない環境でも動くように +09:00 の固定オフセット
    TZ_JST = timezone(timedelta(hours=9))
print(TZ_JST)

Asia/Tokyo


## ロガー設定

In [4]:
### ログ格納用のフォルダの作成
LOG_DIR = "../data/candles/logs"
DATA_DIR = "../data/candles"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

logger = logging.getLogger("data_fetch")
logger.setLevel(logging.INFO)
_handler = logging.StreamHandler()
_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
if not logger.handlers:
    logger.addHandler(_handler)

### Slackログ用
SLACK_LOGGER = logging.getLogger("slack_notify")
SLACK_LOGGER.setLevel(logging.INFO)
_handler = logging.StreamHandler()
_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
if not SLACK_LOGGER.handlers:
    SLACK_LOGGER.addHandler(_handler)

## TRADEディレクトリ設定

In [5]:
TRADES_DIR = "../data/trades"
os.makedirs(TRADES_DIR, exist_ok=True)

## STATEディレクトリ設定

In [6]:
STATE_DIR = "../data/state"
os.makedirs(STATE_DIR, exist_ok=True)

## METRICSディレクトリ設定

In [7]:
METRICS_DIR = "../data/metrics"
os.makedirs(METRICS_DIR, exist_ok=True)

## JSONディレクトリ設定

In [8]:
JSONL_DIR = "../logs/jsonl"
os.makedirs(JSONL_DIR, exist_ok=True)

## CCXTConfig

In [9]:
@dataclass
class CCXTConfig:
    """
    処理:
        ccxt を用いたデータ取得の設定値を保持するデータクラス。fetchOHLCV / fetchTrades の双方に共通。

    INPUT:
        - exchange_id (str): 取引所ID（例: "bitflyer"）
        - symbol (str): 取引ペア（例: "XRP/JPY"）
        - timeframe (str): 取得足の時間粒度（例: "1h"）
        - period_sec (int): 足の秒数（例: 3600）
        - limit (int): 取得する「確定足」本数（例: 200）
        - max_retries (int): 通信リトライ回数
        - retry_backoff_sec (float): リトライ時の指数バックオフ係数
        - timeout_ms (int): HTTPタイムアウト（ms）
        - trades_page_limit (int): fetchTrades の1ページあたりの件数目安（取引所による）
        - api_key / secret (str|None): 実運用で必要なら設定（公開データ取得では不要）

    OUTPUT:
        - なし（設定値の保持のみ）
    """
    exchange_id: str = "binance"
    symbol: str = "XRP/JPY"
    timeframe: str = "1h"
    period_sec: int = 3600
    limit: int = 200
    max_retries: int = 3
    retry_backoff_sec: float = 1.5
    timeout_ms: int = 15000
    trades_page_limit: int = 500
    api_key: Optional[str] = None
    secret: Optional[str] = None

## SignalParams

In [10]:
class SignalParams:
    """
    処理:
        シグナル計算・判定に用いるパラメータ定義（短期/長期のSMA期間、判定の閾値補正など）。
    INPUT:
        - short_window (int): 短期SMAの期間。デフォルト=30
        - long_window (int): 長期SMAの期間。デフォルト=60
        - epsilon (float): SMA比較の際の数値誤差対策（極小値）。
    OUTPUT:
        - なし（設定値の保持のみ）
    """
    short_window: int = 30
    long_window: int = 60
    epsilon: float = 1e-12

## OrderParams

In [11]:
@dataclass
class OrderParams:
    """
    処理:
        注文に関するパラメータを束ねる設定クラス。
        Paper/Realの切替、スリッページ・手数料、1回の発注額（JPY）などを管理。
    INPUT:
        - mode (str): "paper" または "real"
        - notional_jpy (float): 1回のエントリーで使う想定のJPY額（例: 5000）
        - slippage_bps (float): スリッページ（bps指定; 1bps=0.01%）買い時は価格×(1+slip)
        - taker_fee_bps (float): 受け取り手数料（想定; 例: 15=0.15%）
        - api_key / secret (str|None): Realモードで必要な場合に指定
    OUTPUT:
        - なし（設定値の保持のみ）
    """
    mode: str = "paper"
    notional_jpy: float = 500.0
    slippage_bps: float = 5.0
    taker_fee_bps: float = 15.0
    api_key: Optional[str] = None
    secret: Optional[str] = None

## BotState

In [12]:
@dataclass
class BotState:
    """
    処理:
        Bot の取引状態を保持するためのデータモデル。
        JSONと相互変換しやすいよう、プリミティブ型中心で構成。
    INPUT:
        - 初期化時パラメータは任意（未指定はデフォルト値）
    OUTPUT:
        - なし（本クラスは状態保持用）
    """
    position: str = "FLAT"           # "FLAT" or "LONG"
    entry_price: float = 0.0
    size: float = 0.0
    tp: float = 0.0
    sl: float = 0.0
    pnl_cum: float = 0.0             # 累積損益（JPY）
    streak_loss: int = 0             # 連敗カウント
    last_gc_bar_ts: Optional[str] = None  # 直近GCバー（ISO文字列, 重複検知用）
    entry_ts_jst: Optional[str] = None    # 現在の建玉のエントリー時刻（ISO）
    last_updated_jst: Optional[str] = None
    last_daily_summary_date: Optional[str] = None  # "YYYY-MM-DD"（日次サマリ送信用）

## StateStore

In [13]:
class StateStore:
    """
    処理:
        `state.json` の読み書きと、同時実行衝突防止のための簡易ロックを提供する。
        - 原子的保存（temp→置換）で破損防止
        - バックアップ `state.json.bak`
        - ロックファイル `.lock` による排他（簡易）
    INPUT:
        - path (str): JSON保存先のパス（例: "./data/state/state.json"）
    OUTPUT:
        - なし（本クラスは I/O のラッパ）
    """
    def __init__(self, path: str):
        self.path = path
        self.backup_path = f"{path}.bak"
        self.lock_path = f"{path}.lock"
        self._lock_token = None

    def acquire_lock(self, timeout_sec: float = 5.0) -> bool:
        """
        処理:
            ロックファイルを作成し、同時実行を抑止する。
        INPUT:
            - timeout_sec (float): 取得待ちタイムアウト
        OUTPUT:
            - (bool): 取得できれば True。失敗すれば False。
        """
        start = time.time()
        token = str(uuid.uuid4())
        while time.time() - start < timeout_sec:
            try:
                fd = os.open(self.lock_path, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
                with os.fdopen(fd, "w") as f:
                    f.write(token)
                self._lock_token = token
                return True
            except FileExistsError:
                time.sleep(0.1)
        return False

    def release_lock(self) -> None:
        """
        処理:
            自分が保持するロックを解放する（他者のロックは触らない）。
        INPUT:
            - なし
        OUTPUT:
            - なし
        """
        try:
            if not os.path.exists(self.lock_path):
                return
            # 自分のトークンのみ解放
            with open(self.lock_path, "r") as f:
                current = f.read().strip()
            if current == (self._lock_token or ""):
                os.remove(self.lock_path)
        except Exception:
            # ロック解放はベストエフォート
            pass
        finally:
            self._lock_token = None

    def load(self) -> BotState:
        """
        処理:
            `state.json` を読み込んで `BotState` に復元する。存在しなければデフォルトを返す。
        INPUT:
            - なし
        OUTPUT:
            - (BotState): 復元した状態（存在しない場合はデフォルト）
        """
        if not os.path.exists(self.path):
            return BotState(last_updated_jst=datetime.now(TZ_JST).isoformat(timespec="seconds"))
        try:
            with open(self.path, "r", encoding="utf-8") as f:
                d = json.load(f)
            # 不足キーはデフォルトで補完
            base = asdict(BotState())
            base.update(d or {})
            return BotState(**base)
        except Exception as e:
            logger.warning(f"state.json の読込に失敗: {e}. backupからの復元を試みます。")
            if os.path.exists(self.backup_path):
                with open(self.backup_path, "r", encoding="utf-8") as f:
                    d = json.load(f)
                base = asdict(BotState())
                base.update(d or {})
                return BotState(**base)
            # バックアップも無ければデフォルト
            return BotState(last_updated_jst=datetime.now(TZ_JST).isoformat(timespec="seconds"))

    def save(self, state: BotState) -> None:
        """
        処理:
            状態を JSON に **原子的** に保存し、バックアップも更新する。
        INPUT:
            - state (BotState): 保存したい状態
        OUTPUT:
            - なし（失敗時は例外）
        """
        d = asdict(state)
        d["last_updated_jst"] = datetime.now(TZ_JST).isoformat(timespec="seconds")
        tmp = f"{self.path}.tmp"
        with open(tmp, "w", encoding="utf-8") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
        # 既存をバックアップ
        if os.path.exists(self.path):
            shutil.copy2(self.path, self.backup_path)
        # 原子的置換
        os.replace(tmp, self.path)

    def __enter__(self):
        """
        処理:
            `with` 文で使えるようにし、ロック取得 → State 読込を行う。
        INPUT:
            - なし
        OUTPUT:
            - (StateStore): 自身を返す（`store.state` を参照可）
        """
        ok = self.acquire_lock(timeout_sec=5.0)
        if not ok:
            raise RuntimeError("state.json のロック取得に失敗しました。")
        self.state = self.load()
        return self

    def __exit__(self, exc_type, exc, tb):
        """
        処理:
            `with` 文を抜ける際にロックを解放する。
        INPUT/OUTPUT:
            - Pythonのコンテキストマネージャ規約に従う（例外は伝播）
        """
        self.release_lock()

## SlackConfig

In [14]:
@dataclass
class SlackConfig:
    """
    処理:
        Slack Webhook 通知の基本設定を保持するデータクラス。
    INPUT:
        - webhook_url (str|None): Slack Incoming Webhook のURL。None の場合は環境変数から取得を試みる。
        - username (str): 通知に表示するbot名
        - icon_emoji (str): 通知アイコン（:robot_face: など）
        - timeout_sec (int): HTTPタイムアウト
        - max_retries (int): 送信リトライ回数
        - backoff_factor (float): 指数バックオフ係数
    OUTPUT:
        - なし（設定値の保持のみ）
    """
    webhook_url: Optional[str] = None
    username: str = "XRP GC Bot"
    icon_emoji: str = ":robot_face:"
    timeout_sec: int = 10
    max_retries: int = 3
    backoff_factor: float = 1.6

    def resolved_url(self) -> Optional[str]:
        """処理: 優先順位に従ってWebhook URLを返す。INPUT: なし / OUTPUT: str|None"""
        return self.webhook_url or os.getenv("SLACK_WEBHOOK_URL")

In [14]:
# Slack Webhook URL は環境変数 SLACK_WEBHOOK_URL で指定してください。
# 以下は設定例です。実際のシークレットはコードへ記載しないでください。
# os.environ["SLACK_WEBHOOK_URL"] = "https://hooks.slack.com/services/XXXXXXXX/XXXXXXXX/XXXXXXXXXXXXXXXX"


# データ取得

## _floor_to_full_hour_utc

In [15]:
def _floor_to_full_hour_utc(dt_utc: datetime) -> datetime:
    """
    処理:
        与えられたUTC時刻を「直前のちょうどの時刻（分秒を0）」に丸める。
        未確定足（進行中の足）を除外するためのカットオフ基準に利用。

    INPUT:
        - dt_utc (datetime, tz-aware): UTC タイムゾーンの datetime

    OUTPUT:
        - (datetime, tz-aware): 分=0, 秒=0, マイクロ秒=0 に切り下げた UTC 時刻
    """
    return dt_utc.replace(minute=0, second=0, microsecond=0)
print(_floor_to_full_hour_utc(datetime(2025,9,21,11,33,56)))

2025-09-21 11:00:00


## _init_exchange

In [16]:
def _init_exchange(cfg: CCXTConfig):
    """
    処理:
        ccxt の取引所クライアントを初期化し、レートリミット/タイムアウト等を設定する。

    INPUT:
        - cfg (CCXTConfig): 取引所ID, タイムアウト, APIキー等の設定

    OUTPUT:
        - (ccxt.Exchange): 初期化済の取引所クライアントインスタンス
    """
    import ccxt
    klass = getattr(ccxt, cfg.exchange_id)
    exchange = klass({
        "apiKey": cfg.api_key or "",
        "secret": cfg.secret or "",
        "enableRateLimit": True,
        "timeout": cfg.timeout_ms,
    })
    return exchange

print( _init_exchange(CCXTConfig))

Binance


## _fetch_ohlcv_direct

In [17]:
def _fetch_ohlcv_direct(exchange, cfg: CCXTConfig, since_ms: int) -> List[List[Any]]:
    """
    処理:
        取引所が `fetchOHLCV` に対応している場合に、直接 OHLCV を取得する。
        未確定足が混じる可能性があるため、後段でカットオフ処理を行う。

    INPUT:
        - exchange (ccxt.Exchange): 初期化済みの取引所クライアント
        - cfg (CCXTConfig): 取得条件
        - since_ms (int): 取得開始のUNIXミリ秒（過去に遡る起点）

    OUTPUT:
        - (List[List[Any]]): ccxt準拠の OHLCV リスト（[timestamp, open, high, low, close, volume]）
        - 例外: 通信/APIエラーは送り返す（上位でリトライ）
    """
    ohlcv = exchange.fetch_ohlcv(
        symbol=cfg.symbol,
        timeframe=cfg.timeframe,
        since=since_ms,
        limit=cfg.limit + 5  # バッファを持って取得
    )
    return ohlcv

_fetch_ohlcv_direct(_init_exchange(CCXTConfig), CCXTConfig,1)

[[1714464000000, 78.18, 80.54, 78.18, 79.75, 10418.0],
 [1714467600000, 78.94, 78.94, 78.94, 78.94, 38.0],
 [1714471200000, 79.01, 79.31, 79.01, 79.31, 341.0],
 [1714474800000, 78.58, 79.2, 78.18, 79.2, 456.0],
 [1714478400000, 77.07, 77.07, 77.07, 77.07, 3.0],
 [1714482000000, 77.07, 77.07, 77.07, 77.07, 0.0],
 [1714485600000, 77.07, 77.07, 77.07, 77.07, 0.0],
 [1714489200000, 78.62, 89.76, 77.88, 77.88, 72.0],
 [1714492800000, 78.31, 78.31, 78.31, 78.31, 66.0],
 [1714496400000, 78.28, 78.28, 78.28, 78.28, 100.0],
 [1714500000000, 78.28, 78.28, 78.28, 78.28, 0.0],
 [1714503600000, 79.06, 79.06, 79.06, 79.06, 100.0],
 [1714507200000, 79.5, 79.5, 79.5, 79.5, 1170.0],
 [1714510800000, 79.5, 79.5, 79.5, 79.5, 0.0],
 [1714514400000, 79.5, 79.5, 79.5, 79.5, 0.0],
 [1714518000000, 79.5, 79.5, 79.5, 79.5, 0.0],
 [1714521600000, 79.45, 79.45, 79.45, 79.45, 491.0],
 [1714525200000, 79.48, 79.48, 78.92, 79.03, 1554.0],
 [1714528800000, 78.83, 78.83, 78.83, 78.83, 534.0],
 [1714532400000, 79.32, 

##  _fetch_ohlcv_via_trades

In [18]:
def _fetch_ohlcv_via_trades(exchange, cfg: CCXTConfig, since_ms: int, until_ms: int) -> List[List[Any]]:
    """
    処理:
        取引所が `fetchOHLCV` 非対応の場合のフォールバック。
        `fetchTrades` を時間でページングしながら取得し、**1時間バケット**に集計して OHLCV を生成する。

    INPUT:
        - exchange (ccxt.Exchange): 取引所クライアント
        - cfg (CCXTConfig): 設定（symbol, trades_page_limit 等）
        - since_ms (int): 取得開始のUNIXミリ秒
        - until_ms (int): 取得終了のUNIXミリ秒（カットオフ=未確定足直前）

    OUTPUT:
        - (List[List[Any]]): OHLCVリスト [ts(ms), o, h, l, c, v]
        - 注意: 取引が極端に少ない時間帯はバーが生成されない場合あり（後続でreindex）
    """
    all_trades: List[Dict[str, Any]] = []
    cursor = since_ms
    while True:
        trades = exchange.fetch_trades(
            symbol=cfg.symbol,
            since=cursor,
            limit=cfg.trades_page_limit
        )
        if not trades:
            break
        all_trades.extend(trades)
        last_ts = trades[-1]["timestamp"]
        if last_ts >= until_ms:
            break
        # ページング（最後のタイムスタンプ + 1ms から再開）
        cursor = last_ts + 1
        # レートリミット（enableRateLimit有効でも軽くsleep）
        time.sleep(exchange.rateLimit / 1000.0 if hasattr(exchange, "rateLimit") else 0.2)

    if not all_trades:
        return []

    # トレードを DataFrame に詰めて時間バケットでOHLCVへ集計
    tdf = pd.DataFrame([
        {"ts": t["timestamp"], "price": float(t["price"]), "amount": float(t["amount"])}
        for t in all_trades
    ])
    tdf["dt_utc"] = pd.to_datetime(tdf["ts"], unit="ms", utc=True)
    # 1時間バケットキー（床関数）
    bin_seconds = cfg.period_sec
    tdf["bucket_utc"] = (tdf["ts"] // (bin_seconds * 1000)) * (bin_seconds * 1000)

    # 集計
    def ohlc_agg(g: pd.DataFrame) -> pd.Series:
        g_sorted = g.sort_values("ts")
        o = g_sorted["price"].iloc[0]
        h = g["price"].max()
        l = g["price"].min()
        c = g_sorted["price"].iloc[-1]
        v = g["amount"].sum()
        return pd.Series({"open": o, "high": h, "low": l, "close": c, "volume": v})

    odf = tdf.groupby("bucket_utc").apply(ohlc_agg).reset_index()
    odf = odf.rename(columns={"bucket_utc": "timestamp_ms"})
    odf = odf[(odf["timestamp_ms"] >= since_ms) & (odf["timestamp_ms"] <= until_ms)]
    ohlcv = odf[["timestamp_ms", "open", "high", "low", "close", "volume"]].values.tolist()
    return ohlcv

_fetch_ohlcv_via_trades(_init_exchange(CCXTConfig), CCXTConfig, 1, 1)

[]

In [19]:
def fetch_ohlcv_latest_ccxt(cfg: CCXTConfig) -> pd.DataFrame:
    """
    処理:
        - ccxt を用いて bitFlyer の XRP/JPY の **1時間足OHLCV** を「確定足のみ」**最大 cfg.limit 本**取得。
        - 可能なら `fetchOHLCV` を使用、非対応なら `fetchTrades` をページングして **1時間集計**で代替。
        - 取得後、**未確定足を除外**し、**JST列**を付与、**CSV/Parquet** 保存。
        - 保存パスやレコード件数、生成時刻などのメタ情報を `df.attrs["meta"]` に格納。

    INPUT:
        - cfg (CCXTConfig): 取引ペア、足、上限本数、リトライ/タイムアウトなどの設定

    OUTPUT:
        - (pd.DataFrame): インデックス=close_time_jst。主な列：
            ['open_time_jst','close_time_jst','open','high','low','close','volume',
             'open_time_utc','close_time_utc','timestamp_ms']
          `df.attrs["meta"]` には各種メタ情報を格納。
        - 例外: ネットワーク/APIエラー時は RuntimeError / ValueError を送出
    """
    # カットオフ（未確定足除外用）= 直前の「ちょうどの時」UTC
    now_utc = datetime.now(TZ_UTC)
    cutoff_utc = _floor_to_full_hour_utc(now_utc)
    cutoff_ms = int(cutoff_utc.timestamp() * 1000)

    # 取りたい本数 + α を過去に遡る
    need = cfg.limit + 5
    span_ms = need * cfg.period_sec * 1000
    since_ms = cutoff_ms - span_ms

    # 取引所初期化
    exchange = _init_exchange(cfg)

    # マーケットロード（シンボル検証）
    last_err = None
    for attempt in range(1, cfg.max_retries + 1):
        try:
            exchange.load_markets()
            break
        except Exception as e:
            last_err = e
            time.sleep(cfg.retry_backoff_sec ** (attempt - 1))
    else:
        raise RuntimeError(f"load_markets failed after {cfg.max_retries} retries: {last_err}")

    if cfg.symbol not in exchange.symbols:
        raise ValueError(f"Symbol {cfg.symbol} not found on {cfg.exchange_id}.")

    # fetchOHLCV 直接 or fetchTrades フォールバック
    ohlcv_rows: List[List[Any]] = []
    if getattr(exchange, "has", {}).get("fetchOHLCV", False):
        for attempt in range(1, cfg.max_retries + 1):
            try:
                ohlcv_rows = _fetch_ohlcv_direct(exchange, cfg, since_ms)
                break
            except Exception as e:
                last_err = e
                time.sleep(cfg.retry_backoff_sec ** (attempt - 1))
        else:
            raise RuntimeError(f"fetch_ohlcv failed after {cfg.max_retries} retries: {last_err}")
    else:
        for attempt in range(1, cfg.max_retries + 1):
            try:
                ohlcv_rows = _fetch_ohlcv_via_trades(exchange, cfg, since_ms, cutoff_ms)
                break
            except Exception as e:
                last_err = e
                time.sleep(cfg.retry_backoff_sec ** (attempt - 1))
        else:
            raise RuntimeError(f"fetch_trades->ohlcv failed after {cfg.max_retries} retries: {last_err}")

    if not ohlcv_rows:
        raise ValueError("No OHLCV rows acquired.")

    # DataFrameへ正規化
    df = pd.DataFrame(ohlcv_rows, columns=["timestamp_ms", "open", "high", "low", "close", "volume"])

    # 未確定足を除外（cutoff_ms より新しいバーが来る実装もあるため）
    df = df[df["timestamp_ms"] <= cutoff_ms]

    # 最新側から limit 本にトリム（昇順に揃える）
    df = df.sort_values("timestamp_ms")
    if len(df) > cfg.limit:
        df = df.iloc[-cfg.limit:].copy()

    # 時刻カラムをUTC/JSTで生成
    df["close_time_utc"] = pd.to_datetime(df["timestamp_ms"], unit="ms", utc=True)
    df["open_time_utc"] = df["close_time_utc"] - pd.to_timedelta(cfg.period_sec, unit="s")
    df["close_time_jst"] = df["close_time_utc"].dt.tz_convert(TZ_JST)
    df["open_time_jst"] = df["open_time_utc"].dt.tz_convert(TZ_JST)

    # インデックス設定＆列順
    df = df.set_index("close_time_jst")
    use_cols = [
        "open_time_jst", "open", "high", "low", "close", "volume",
        "open_time_utc", "close_time_utc", "timestamp_ms",
    ]
    df = df[use_cols]

    # 保存（CSVローテ / Parquet最新）
    ts_label = datetime.now(TZ_JST).strftime("%Y%m%d_%H%M%S")
    csv_path = os.path.join(LOG_DIR, f"xrpjpy_1h_{ts_label}.csv")
    df.to_csv(csv_path, encoding="utf-8")
    pq_path = os.path.join(DATA_DIR, "xrpjpy_1h_latest.parquet")

    parquet_ok = False
    try:
        import pyarrow  # noqa: F401
        df.to_parquet(pq_path, index=True)
        parquet_ok = True
    except Exception as e:
        logger.warning(f"Parquet save skipped (pyarrow未導入 or 失敗): {e}")

    # メタ情報
    df.attrs["meta"] = {
        "exchange": cfg.exchange_id,
        "symbol": cfg.symbol,
        "timeframe": cfg.timeframe,
        "period_sec": cfg.period_sec,
        "limit": cfg.limit,
        "rows": len(df),
        "csv_path": csv_path,
        "parquet_path": pq_path if parquet_ok else None,
        "cutoff_utc": cutoff_utc.isoformat(),
        "generated_at_jst": datetime.now(TZ_JST).isoformat(timespec="seconds"),
        "used_method": "fetchOHLCV" if getattr(_init_exchange(cfg), "has", {}).get("fetchOHLCV", False) else "trades_aggregate"
    }
    return df

fetch_ohlcv_latest_ccxt(CCXTConfig)

,open_time_jst,open,high,low,close,volume,open_time_utc,close_time_utc,timestamp_ms
close_time_jst,,,,,,,,,
2025-09-19 13:00:00+09:00,2025-09-19 12:00:00+09:00,450.91,451.00,446.59,451.00,125922.0,2025-09-19 03:00:00+00:00,2025-09-19 04:00:00+00:00,1758254400000
2025-09-19 14:00:00+09:00,2025-09-19 13:00:00+09:00,450.92,451.53,449.24,450.85,45081.7,2025-09-19 04:00:00+00:00,2025-09-19 05:00:00+00:00,1758258000000
2025-09-19 15:00:00+09:00,2025-09-19 14:00:00+09:00,450.84,450.84,448.61,448.77,118918.3,2025-09-19 05:00:00+00:00,2025-09-19 06:00:00+00:00,1758261600000
2025-09-19 16:00:00+09:00,2025-09-19 15:00:00+09:00,448.76,450.98,448.44,450.60,75779.5,2025-09-19 06:00:00+00:00,2025-09-19 07:00:00+00:00,1758265200000
2025-09-19 17:00:00+09:00,2025-09-19 16:00:00+09:00,450.59,452.00,450.06,450.11,54109.0,2025-09-19 07:00:00+00:00,2025-09-19 08:00:00+00:00,1758268800000
...,...,...,...,...,...,...,...,...,...
2025-09-27 16:00:00+09:00,2025-09-27 15:00:00+09:00,419.32,420.00,416.78,416.79,41173.6,2025-09-27 06:00:00+00:00,2025-09-27 07:00:00+00:00,1758956400000
2025-09-27 17:00:00+09:00,2025-09-27 16:00:00+09:00,416.87,416.92,414.95,415.08,43128.9,2025-09-27 07:00:00+00:00,2025-09-27 08:00:00+00:00,1758960000000
2025-09-27 18:00:00+09:00,2025-09-27 17:00:00+09:00,415.10,417.07,414.07,416.74,31636.4,2025-09-27 08:00:00+00:00,2025-09-27 09:00:00+00:00,1758963600000


## load_latest_cached_ccxt

In [20]:
def load_latest_cached_ccxt() -> pd.DataFrame:
    """
    処理:
        ccxt版の保存先（`./data/candles/xrpjpy_1h_latest.parquet`）を優先して読み込み、
        無ければ `./data/candles/logs/` の最新CSVをロードする。

    INPUT:
        - なし（保存先ディレクトリを既定パスから探索）

    OUTPUT:
        - (pd.DataFrame): `fetch_ohlcv_latest_ccxt` と同じ列構成の DataFrame
        - 例外: キャッシュが存在しない場合は FileNotFoundError
    """
    pq_path = os.path.join(DATA_DIR, "xrpjpy_1h_latest.parquet")
    if os.path.exists(pq_path):
        return pd.read_parquet(pq_path)

    csv_files = sorted(
        [f for f in os.listdir(LOG_DIR) if f.startswith("xrpjpy_1h_") and f.endswith(".csv")]
    )
    if not csv_files:
        raise FileNotFoundError("No cached parquet or CSV logs found.")
    latest_csv = os.path.join(LOG_DIR, csv_files[-1])
    return pd.read_csv(
        latest_csv,
        parse_dates=["open_time_utc", "close_time_utc", "open_time_jst", "close_time_jst"],
        index_col="close_time_jst"
    )

load_latest_cached_ccxt()

,open_time_jst,open,high,low,close,volume,open_time_utc,close_time_utc,timestamp_ms
close_time_jst,,,,,,,,,
2025-09-19 13:00:00+09:00,2025-09-19 12:00:00+09:00,450.91,451.00,446.59,451.00,125922.0,2025-09-19 03:00:00+00:00,2025-09-19 04:00:00+00:00,1758254400000
2025-09-19 14:00:00+09:00,2025-09-19 13:00:00+09:00,450.92,451.53,449.24,450.85,45081.7,2025-09-19 04:00:00+00:00,2025-09-19 05:00:00+00:00,1758258000000
2025-09-19 15:00:00+09:00,2025-09-19 14:00:00+09:00,450.84,450.84,448.61,448.77,118918.3,2025-09-19 05:00:00+00:00,2025-09-19 06:00:00+00:00,1758261600000
2025-09-19 16:00:00+09:00,2025-09-19 15:00:00+09:00,448.76,450.98,448.44,450.60,75779.5,2025-09-19 06:00:00+00:00,2025-09-19 07:00:00+00:00,1758265200000
2025-09-19 17:00:00+09:00,2025-09-19 16:00:00+09:00,450.59,452.00,450.06,450.11,54109.0,2025-09-19 07:00:00+00:00,2025-09-19 08:00:00+00:00,1758268800000
...,...,...,...,...,...,...,...,...,...
2025-09-27 16:00:00+09:00,2025-09-27 15:00:00+09:00,419.32,420.00,416.78,416.79,41173.6,2025-09-27 06:00:00+00:00,2025-09-27 07:00:00+00:00,1758956400000
2025-09-27 17:00:00+09:00,2025-09-27 16:00:00+09:00,416.87,416.92,414.95,415.08,43128.9,2025-09-27 07:00:00+00:00,2025-09-27 08:00:00+00:00,1758960000000
2025-09-27 18:00:00+09:00,2025-09-27 17:00:00+09:00,415.10,417.07,414.07,416.74,31636.4,2025-09-27 08:00:00+00:00,2025-09-27 09:00:00+00:00,1758963600000


## お試し

In [21]:
# === 使い方（実行例・オンライン環境で） ===
if __name__ == "__main__":
    cfg = CCXTConfig(limit=200)
    try:
        df = fetch_ohlcv_latest_ccxt(cfg)
        meta = df.attrs.get("meta", {})
        print(f"Fetched {meta.get('rows')} rows via {meta.get('used_method')}.")
        print(f"Saved CSV -> {meta.get('csv_path')}")
        if meta.get("parquet_path"):
            print(f"Saved Parquet -> {meta.get('parquet_path')}")
        display(df.tail(5))
    except Exception as e:
        logger.exception(f"Data fetch error: {e}")
        print("取得に失敗した場合は、キャッシュ読込を試してください：\n"
              "df_cached = load_latest_cached_ccxt(); display(df_cached.tail(5))")

Fetched 200 rows via fetchOHLCV.
Saved CSV -> ../data/candles/logs/xrpjpy_1h_20250927_212918.csv
Saved Parquet -> ../data/candles/xrpjpy_1h_latest.parquet


,open_time_jst,open,high,low,close,volume,open_time_utc,close_time_utc,timestamp_ms
close_time_jst,,,,,,,,,
2025-09-27 16:00:00+09:00,2025-09-27 15:00:00+09:00,419.32,420.00,416.78,416.79,41173.6,2025-09-27 06:00:00+00:00,2025-09-27 07:00:00+00:00,1758956400000
2025-09-27 17:00:00+09:00,2025-09-27 16:00:00+09:00,416.87,416.92,414.95,415.08,43128.9,2025-09-27 07:00:00+00:00,2025-09-27 08:00:00+00:00,1758960000000
2025-09-27 18:00:00+09:00,2025-09-27 17:00:00+09:00,415.10,417.07,414.07,416.74,31636.4,2025-09-27 08:00:00+00:00,2025-09-27 09:00:00+00:00,1758963600000
2025-09-27 19:00:00+09:00,2025-09-27 18:00:00+09:00,416.80,418.64,416.52,418.00,40235.9,2025-09-27 09:00:00+00:00,2025-09-27 10:00:00+00:00,1758967200000
2025-09-27 20:00:00+09:00,2025-09-27 19:00:00+09:00,417.96,417.96,416.07,416.30,24945.2,2025-09-27 10:00:00+00:00,2025-09-27 11:00:00+00:00,1758970800000


# シグナル作成

## add_sma_columns()

In [22]:
def add_sma_columns(df: pd.DataFrame, params: SignalParams) -> pd.DataFrame:
    """
    処理:
        与えられたローソク足DataFrameに、短期/長期SMA列を追加して返す。
        対象は close 列。long_window 本以上のデータがなければ例外を送出。
    INPUT:
        - df (pd.DataFrame): インデックス=時間（推奨: close_time_jst）。少なくとも 'close' 列を含むこと。
        - params (SignalParams): 短期/長期の期間設定等。
    OUTPUT:
        - (pd.DataFrame): 'sma_short', 'sma_long' を追加した DataFrame（元の参照とは別オブジェクト）。
        - 例外:
            - ValueError: データ本数不足（len(df) < long_window）
            - KeyError: 'close' 列が存在しない
    """
    if "close" not in df.columns:
        raise KeyError("DataFrameに 'close' 列がありません。")

    if len(df) < params.long_window:
        raise ValueError(f"Not enough bars to compute SMA: need >= {params.long_window}, got {len(df)}.")

    out = df.copy()
    out["sma_short"] = out["close"].rolling(window=params.short_window, min_periods=params.short_window).mean()
    out["sma_long"]  = out["close"].rolling(window=params.long_window,  min_periods=params.long_window).mean()
    return out

add_sma_columns(df, SignalParams).tail(3)

,open_time_jst,open,high,low,close,volume,open_time_utc,close_time_utc,timestamp_ms,sma_short,sma_long
close_time_jst,,,,,,,,,,,
2025-09-27 18:00:00+09:00,2025-09-27 17:00:00+09:00,415.10,417.07,414.07,416.74,31636.4,2025-09-27 08:00:00+00:00,2025-09-27 09:00:00+00:00,1758963600000,415.128000,418.593833
2025-09-27 19:00:00+09:00,2025-09-27 18:00:00+09:00,416.80,418.64,416.52,418.00,40235.9,2025-09-27 09:00:00+00:00,2025-09-27 10:00:00+00:00,1758967200000,415.232000,418.252000
2025-09-27 20:00:00+09:00,2025-09-27 19:00:00+09:00,417.96,417.96,416.07,416.30,24945.2,2025-09-27 10:00:00+00:00,2025-09-27 11:00:00+00:00,1758970800000,415.321333,417.924833


## detect_golden_cross_lates()

In [23]:
def detect_golden_cross_latest(
    df_with_sma: pd.DataFrame,
    params: SignalParams,
    last_signaled_bar_ts: Optional[pd.Timestamp] = None
) -> Dict[str, Any]:
    """
    処理:
        直近の確定足（= DataFrame の末尾行）で **ゴールデンクロス(GC)** が成立したかを判定する。
        判定条件は「直前バーで `sma_short <= sma_long` かつ 直近バーで `sma_short > sma_long`」。
        同一バーでの多重検知を避けるため、前回シグナルのバー時刻 `last_signaled_bar_ts` を渡すと、
        それと一致するバーでは `already_signaled=True` を立てて発報を抑止できる。
    INPUT:
        - df_with_sma (pd.DataFrame): 'sma_short','sma_long','close' を含む時系列。末尾が直近確定足。
        - params (SignalParams): 判定パラメータ（期間/epsilon）
        - last_signaled_bar_ts (pd.Timestamp|None): 直近でシグナル発報済みのバーの時刻（JSTインデックス想定）。
    OUTPUT:
        - (dict): シグナル判定結果
            {
              "is_gc": bool,                 # 今回バーでGCが成立したか
              "already_signaled": bool,      # 同一バーの重複か（Trueなら通知/発注は抑止）
              "bar_ts": pd.Timestamp,        # 今回の判定対象バー（末尾=直近確定足）
              "price": float,                # 判定バーの終値
              "sma_short": float,            # 判定バー時点の短期SMA
              "sma_long": float,             # 判定バー時点の長期SMA
              "prev_sma_short": float,       # 直前バー時点の短期SMA
              "prev_sma_long": float,        # 直前バー時点の長期SMA
            }
        - 例外:
            - ValueError: SMAの欠損（計算対象不足）
    """
    if any(col not in df_with_sma.columns for col in ["sma_short", "sma_long", "close"]):
        raise ValueError("df_with_sma には 'sma_short', 'sma_long', 'close' が必要です。")

    # 末尾が直近確定足
    if len(df_with_sma) < 2:
        raise ValueError("クロス判定には少なくとも2本のバーが必要です。")

    # 末尾と直前の行を取得
    latest = df_with_sma.iloc[-1]
    prev   = df_with_sma.iloc[-2]
    bar_ts = df_with_sma.index[-1]

    # SMAが未計算（NaN）の場合は判定できない
    if pd.isna(latest["sma_short"]) or pd.isna(latest["sma_long"]) or pd.isna(prev["sma_short"]) or pd.isna(prev["sma_long"]):
        raise ValueError("SMAがNaNです。データ本数が足りているか確認してください。")

    eps = params.epsilon
    crossed_up = (prev["sma_short"] <= prev["sma_long"] + eps) and (latest["sma_short"] > latest["sma_long"] + eps)

    already = False
    if last_signaled_bar_ts is not None and pd.to_datetime(last_signaled_bar_ts) == pd.to_datetime(bar_ts):
        # 同じバーでは重複検知扱い
        already = True

    return {
        "is_gc": bool(crossed_up),
        "already_signaled": bool(already),
        "bar_ts": bar_ts,
        "price": float(latest["close"]),
        "sma_short": float(latest["sma_short"]),
        "sma_long": float(latest["sma_long"]),
        "prev_sma_short": float(prev["sma_short"]),
        "prev_sma_long": float(prev["sma_long"]),
    }

detect_golden_cross_latest(
    add_sma_columns(df, SignalParams),
    SignalParams,
)

{'is_gc': False,
 'already_signaled': False,
 'bar_ts': Timestamp('2025-09-27 20:00:00+0900', tz='Asia/Tokyo'),
 'price': 416.3,
 'sma_short': 415.32133333333337,
 'sma_long': 417.92483333333337,
 'prev_sma_short': 415.23199999999997,
 'prev_sma_long': 418.25200000000007}

## update_state_after_signal

In [24]:
def update_state_after_signal(state: Dict[str, Any], signal: Dict[str, Any]) -> Dict[str, Any]:
    """
    処理:
        シグナル発生後に `state` を更新するヘルパー。
        ここでは多重検知防止のための `last_gc_bar_ts`（最後にGCが発生したバー時刻）を更新する。
        ※ 将来的に position/entry_price/tp/sl などもここで管理可能。
    INPUT:
        - state (dict): 既存の状態（なければ空dictでも可）。
        - signal (dict): detect_golden_cross_latest() の戻り値。
    OUTPUT:
        - (dict): 更新済みの state 辞書。
            追記/更新されるキー:
              - "last_gc_bar_ts": pd.Timestamp (ISO文字列で保存するのが推奨)
    """
    new_state = dict(state or {})
    if signal.get("is_gc", False):
        # 保管はISO文字列にしておくとJSON保存しやすい
        ts = signal["bar_ts"]
        if hasattr(ts, "isoformat"):
            ts = ts.isoformat()
        new_state["last_gc_bar_ts"] = ts
    return new_state

## お試し

In [25]:
if __name__ == "__main__":
    try:
        # 1) データ（前段の取得ステップの結果）を読み込む
        from pathlib import Path
        # 優先: 最新キャッシュ（ccxt版）
        if (Path("../data/candles/xrpjpy_1h_latest.parquet")).exists():
            df_src = pd.read_parquet("../data/candles/xrpjpy_1h_latest.parquet")
            # display(df_src)
        else:
            # ログCSVの最新を拾う（簡易）
            logs = sorted([p for p in Path("../data/candles/logs").glob("xrpjpy_1h_*.csv")])
            if not logs:
                raise FileNotFoundError("取得データが見つかりません。先にデータ取得ステップを実行してください。")
            df_src = pd.read_csv(logs[-1],
                                 parse_dates=["open_time_utc", "close_time_utc", "open_time_jst", "close_time_jst"],
                                 index_col="close_time_jst")

        # 2) SMA列を追加
        params = SignalParams()
        df_feat = add_sma_columns(df_src, params)

        # 3) 直近確定足でのGC判定（last_signaled_bar_tsは state.json から読む想定）
        last_signaled_bar_ts = None  # 例: ISO文字列 "2025-09-19T22:00:00+09:00"
        sig = detect_golden_cross_latest(df_feat, params, last_signaled_bar_ts=last_signaled_bar_ts)

        # 4) 結果表示（このセルはノートブック確認用）
        print("=== Signal Result ===")
        for k, v in sig.items():
            print(f"{k:>18}: {v}")

        # 5) state 更新例（実運用では JSON に保存）
        state = {}
        state = update_state_after_signal(state, sig)
        print("\nUpdated state:", state)

        # 6) ビュー用途：末尾5行だけ確認
        display(df_feat.tail(5))

    except Exception as e:
        print("Signal step error:", e)

=== Signal Result ===
             is_gc: False
  already_signaled: False
            bar_ts: 2025-09-27 20:00:00+09:00
             price: 416.3
         sma_short: 415.32133333333337
          sma_long: 417.92483333333337
    prev_sma_short: 415.23199999999997
     prev_sma_long: 418.25200000000007

Updated state: {}


,open_time_jst,open,high,low,close,volume,open_time_utc,close_time_utc,timestamp_ms,sma_short,sma_long
close_time_jst,,,,,,,,,,,
2025-09-27 16:00:00+09:00,2025-09-27 15:00:00+09:00,419.32,420.00,416.78,416.79,41173.6,2025-09-27 06:00:00+00:00,2025-09-27 07:00:00+00:00,1758956400000,415.038000,419.420333
2025-09-27 17:00:00+09:00,2025-09-27 16:00:00+09:00,416.87,416.92,414.95,415.08,43128.9,2025-09-27 07:00:00+00:00,2025-09-27 08:00:00+00:00,1758960000000,415.056667,418.960167
2025-09-27 18:00:00+09:00,2025-09-27 17:00:00+09:00,415.10,417.07,414.07,416.74,31636.4,2025-09-27 08:00:00+00:00,2025-09-27 09:00:00+00:00,1758963600000,415.128000,418.593833
2025-09-27 19:00:00+09:00,2025-09-27 18:00:00+09:00,416.80,418.64,416.52,418.00,40235.9,2025-09-27 09:00:00+00:00,2025-09-27 10:00:00+00:00,1758967200000,415.232000,418.252000
2025-09-27 20:00:00+09:00,2025-09-27 19:00:00+09:00,417.96,417.96,416.07,416.30,24945.2,2025-09-27 10:00:00+00:00,2025-09-27 11:00:00+00:00,1758970800000,415.321333,417.924833


# 注文処理

## _now_jst()

In [26]:
def _now_jst() -> datetime:
    """処理: 現在時刻(JST)を返す。INPUT: なし / OUTPUT: datetime(tz-aware)"""
    return datetime.now(TZ_JST)
print(_now_jst())

2025-09-27 21:29:18.447493+09:00


## compute_tp_sl()

In [27]:
def compute_tp_sl(entry_price: float, tp_pct: float = 0.02, sl_pct: float = 0.03) -> Tuple[float, float]:
    """
    処理:
        ロング前提で利確(TP)・損切り(SL)水準を算出する。
    INPUT:
        - entry_price (float): 約定価格
        - tp_pct (float): 利確率（例: 0.02 = +2%）
        - sl_pct (float): 損切率（例: 0.03 = -3%）
    OUTPUT:
        - (tp, sl) (Tuple[float, float]): 価格水準
    """
    tp = entry_price * (1.0 + tp_pct)
    sl = entry_price * (1.0 - sl_pct)
    return (float(tp), float(sl))

## _init_ccxt_for_real

In [28]:
def _init_ccxt_for_real(api_key: Optional[str], secret: Optional[str]):
    """
    処理:
        Realモードのために ccxt の bitFlyer クライアントを初期化する。
    INPUT:
        - api_key (str|None), secret (str|None): 発注に必要であれば指定
    OUTPUT:
        - exchange (ccxt.Exchange): 初期化済みクライアント
    """
    import ccxt
    return ccxt.bitflyer({
        "apiKey": api_key or "",
        "secret": secret or "",
        "enableRateLimit": True,
        "timeout": 15000,
    })

# 実運用時の利用例:
# from os import getenv
# exchange_for_real = _init_ccxt_for_real(
#     api_key=getenv("BITFLYER_API_KEY"),
#     secret=getenv("BITFLYER_API_SECRET")
# )
# exchange_for_real.load_markets()
# print(exchange_for_real)


bitFlyer


## _fit_amount_to_market()

In [29]:
def _fit_amount_to_market(exchange, symbol: str, amount: float) -> float:
    """
    処理:
        取引所の最小数量・刻みに合わせて数量を調整（切り捨て）。
    INPUT:
        - exchange (ccxt.Exchange): `load_markets()` 済のクライアント
        - symbol (str): 例 "XRP/JPY"
        - amount (float): 希望数量（XRPの枚数）
    OUTPUT:
        - (float): 取引所の制約に合わせた数量（0になる場合は発注不可）
    """
    m = exchange.market(symbol)
    min_amt = (m.get("limits", {}).get("amount", {}) or {}).get("min", 0.0) or 0.0
    step = (m.get("precision", {}) or {}).get("amount", None)
    # precision は小数桁数を表すことが多い（例: 2 → 0.01刻み）
    if step is not None and isinstance(step, int):
        quant = 10 ** step
        adj = math.floor(amount * quant) / quant
    else:
        # precision 情報がない場合は min のみ尊重して丸め
        adj = amount
    if min_amt and adj < min_amt:
        return 0.0
    return float(adj)

_fit_amount_to_market(exchange_for_real, 
                      symbol="XRP/JPY", 
                      amount=0.00001)

1e-05

## decide_order_size_jpy_to_amount()

In [30]:
def decide_order_size_jpy_to_amount(price: float, notional_jpy: float) -> float:
    """
    処理:
        指定した JPY の発注額を、与えられた価格に基づいて「数量（XRP）」に変換する。
    INPUT:
        - price (float): 市場の参照価格（closeなど）
        - notional_jpy (float): JPY建ての発注額
    OUTPUT:
        - (float): 数量（XRP枚数）
    """
    if price <= 0:
        raise ValueError("price must be positive.")
    return float(notional_jpy / price)

## _ensure_tradelog

In [31]:
def _ensure_tradelog():
    """処理: trades.csv の存在を保証（無ければヘッダ付きで作成）。INPUT: なし / OUTPUT: なし"""
    path = os.path.join(TRADES_DIR, "trades.csv")
    if not os.path.exists(path):
        cols = [
            "ts_jst","mode","symbol","side","size","price","notional_jpy",
            "fee_jpy","slippage_bps","taker_fee_bps","tp","sl","order_id","raw"
        ]
        pd.DataFrame(columns=cols).to_csv(path, index=False, encoding="utf-8")

## _append_trade_row()

In [32]:
def _append_trade_row(row: Dict[str, Any]):
    """処理: trades.csv に1行追記する。INPUT: 追記データ / OUTPUT: なし"""
    _ensure_tradelog()
    path = os.path.join(TRADES_DIR, "trades.csv")
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False, encoding="utf-8")

## place_market_buy()

In [33]:
def place_market_buy(
    symbol: str,
    ref_price: float,
    params: OrderParams,
    exchange_for_real: Optional[Any] = None
) -> Dict[str, Any]:
    """
    処理:
        ロングエントリーの **成行買い** を実行する。
        - Paper: スリッページ・手数料を加味して内部約定
        - Real : ccxt(bitFlyer)で `create_order(..., type='market', side='buy')`
        いずれも **trades.csv** にログを書き出す。
    INPUT:
        - symbol (str): 例 "XRP/JPY"
        - ref_price (float): 参照価格（通常は直近確定足の終値）
        - params (OrderParams): mode, notional, slippage/taker_fee など
        - exchange_for_real (ccxt.Exchange|None): 事前に `load_markets()` まで済ませて渡すと効率的
    OUTPUT:
        - (dict): 約定結果（共通フィールド）
            {
              "mode","symbol","side","size","price","notional_jpy",
              "fee_jpy","tp","sl","order_id","raw"
            }
        - 例外: RealモードでのAPIエラー等は例外送出
    """
    ### 現在時刻の取得
    ts_jst = _now_jst().isoformat(timespec="seconds")
    side = "buy"

    # --- 注文数量計算（JPY→数量） ---
    size = decide_order_size_jpy_to_amount(ref_price, params.notional_jpy)
    print(f"size:{size}")

    if params.mode == "paper":
        # Paper fill: スリッページ・手数料適用
        slip = params.slippage_bps / 10000.0
        fee = params.taker_fee_bps / 10000.0
        fill_price = ref_price * (1.0 + slip)
        notional = fill_price * size
        fee_jpy = notional * fee
        order_id = f"PAPER-{int(time.time())}"
        raw = {"reason": "paper_fill", "ref_price": ref_price}

    elif params.mode == "real":
        # Real fill: ccxtに発注し、取引所仕様にサイズを合わせる
        if exchange_for_real is None:
            exchange_for_real = _init_ccxt_for_real(params.api_key, params.secret)
        exchange_for_real.load_markets()

        adj_size = _fit_amount_to_market(exchange_for_real, symbol, size)
        if adj_size <= 0:
            raise ValueError("Calculated order size is below exchange minimum/precision.")

        # bitFlyerは JPY 表示だが ccxt経由で市場成行を呼べる
        order = exchange_for_real.create_order(symbol=symbol, type="market", side="buy", amount=adj_size)
        order_id = order.get("id")
        # 約定価格の推定（多くの取引所は filled/price に入るが未確定なら平均を推定）
        filled = float(order.get("filled", adj_size))
        avg = float(order.get("average") or order.get("price") or ref_price)
        fill_price = avg
        size = filled  # 実際に約定したサイズ
        notional = fill_price * size
        # 手数料（取得できない場合はパラメータの想定値で計算）
        fee_cost = None
        if order.get("fees"):
            try:
                # JPY建てのfeeを特定（無ければ最初の要素を採用）
                fee_obj = order["fees"][0]
                fee_cost = float(fee_obj.get("cost", 0.0))
            except Exception:
                fee_cost = None
        if fee_cost is None:
            fee_cost = notional * (params.taker_fee_bps / 10000.0)
        fee_jpy = fee_cost
        raw = order

    else:
        raise ValueError("OrderParams.mode must be 'paper' or 'real'.")

    # TP/SL計算（ロング前提）
    tp, sl = compute_tp_sl(fill_price, tp_pct=0.02, sl_pct=0.03)

    # ログ行
    row = {
        "ts_jst": ts_jst,
        "mode": params.mode,
        "symbol": symbol,
        "side": side,
        "size": round(size, 8),
        "price": round(fill_price, 8),
        "notional_jpy": round(notional, 2),
        "fee_jpy": round(fee_jpy, 2),
        "slippage_bps": params.slippage_bps,
        "taker_fee_bps": params.taker_fee_bps,
        "tp": round(tp, 8),
        "sl": round(sl, 8),
        "order_id": order_id,
        "raw": json.dumps(raw, ensure_ascii=False),
    }
    _append_trade_row(row)

    return {
        "mode": params.mode,
        "symbol": symbol,
        "side": side,
        "size": float(row["size"]),
        "price": float(row["price"]),
        "notional_jpy": float(row["notional_jpy"]),
        "fee_jpy": float(row["fee_jpy"]),
        "tp": float(row["tp"]),
        "sl": float(row["sl"]),
        "order_id": order_id,
        "raw": raw,
    }

## should_open_from_signal

In [34]:
def should_open_from_signal(state: Dict[str, Any], signal: Dict[str, Any]) -> bool:
    """
    処理:
        現在の状態とシグナルを見て、**新規エントリーすべきか**を判定する。
        - いまのポジションが FLAT
        - signal["is_gc"] が True
        - signal["already_signaled"] が False（多重検知抑止）
    INPUT:
        - state (dict): 例 {"position":"FLAT", ...} など。未設定なら FLAT とみなす実装を推奨。
        - signal (dict): detect_golden_cross_latest() の戻り値
    OUTPUT:
        - (bool): エントリーするなら True
    """
    pos = (state or {}).get("position", "FLAT")
    return (pos == "FLAT") and bool(signal.get("is_gc")) and (not bool(signal.get("already_signaled")))

## update_state_on_entry

In [35]:
def update_state_on_entry(state: Dict[str, Any], order_result: Dict[str, Any]) -> Dict[str, Any]:
    """
    処理:
        約定結果に基づき、取引状態を **LONG** に更新する。
        エントリー価格・数量・TP/SL・累積損益（未設定なら0）などを記録する。
    INPUT:
        - state (dict): 既存の状態（JSONから読み込む想定）。
        - order_result (dict): place_market_buy() の戻り値。
    OUTPUT:
        - (dict): 更新済み状態。
            追記/更新される主なキー:
              position="LONG", entry_price, size, tp, sl, pnl_cum
    """
    new_state = dict(state or {})
    new_state["position"] = "LONG"
    new_state["entry_price"] = float(order_result["price"])
    new_state["size"] = float(order_result["size"])
    new_state["tp"] = float(order_result["tp"])
    new_state["sl"] = float(order_result["sl"])
    new_state["pnl_cum"] = float(new_state.get("pnl_cum", 0.0))
    # 日次等のログ用に、エントリーのタイムスタンプも残す
    new_state["entry_ts_jst"] = _now_jst().isoformat(timespec="seconds")
    return new_state

## お試し

In [36]:
if __name__ == "__main__":
    try:
        # 0) ここではシグナルを疑似的に用意（実際は Step 2 の detect_golden_cross_latest の結果を使用）
        mock_signal = {
            "is_gc": True,
            "already_signaled": False,
            "bar_ts": pd.Timestamp("2025-09-20T12:00:00+09:00"),
            "price": 100.0,  # 例としての参考価格
            "sma_short": 99.5,
            "sma_long": 99.4,
            "prev_sma_short": 99.3,
            "prev_sma_long": 99.4,
        }
        state = {"position": "FLAT", "pnl_cum": 0.0}

        if should_open_from_signal(state, mock_signal):
            params = OrderParams(
                mode="paper",            # 実運用で切り替え: "real"
                notional_jpy=500.0,
                slippage_bps=5.0,
                taker_fee_bps=15.0,
                api_key=None, secret=None
            )

            # Realモードの場合は、事前に ccxt を初期化して渡すと効率的
            exchange = None
            from ccxt import bitflyer; exchange = bitflyer({"apiKey":" ",
                                                            "secret":"",
                                                            "enableRateLimit": True})

            order_res = place_market_buy("XRP/JPY", mock_signal["price"], params, exchange_for_real=exchange)
            state = update_state_on_entry(state, order_res)

            print("Order Result:", order_res)
            print("Updated State:", state)
        else:
            print("No Entry (条件未充足)")

    except Exception as e:
        logger.exception(f"Order step error: {e}")

size:5.0
Order Result: {'mode': 'paper', 'symbol': 'XRP/JPY', 'side': 'buy', 'size': 5.0, 'price': 100.05, 'notional_jpy': 500.25, 'fee_jpy': 0.75, 'tp': 102.051, 'sl': 97.0485, 'order_id': 'PAPER-1758976160', 'raw': {'reason': 'paper_fill', 'ref_price': 100.0}}
Updated State: {'position': 'LONG', 'pnl_cum': 0.0, 'entry_price': 100.05, 'size': 5.0, 'tp': 102.051, 'sl': 97.0485, 'entry_ts_jst': '2025-09-27T21:29:20+09:00'}


# ポジション管理

## ensure_single_position()

In [37]:
def ensure_single_position(state: BotState) -> None:
    """
    処理:
        state が単一建玉の制約を満たしているか軽く検証し、異常なら例外。
    INPUT:
        - state (BotState): 検証対象
    OUTPUT:
        - なし（異常時は ValueError 例外）
    """
    if state.position not in ("FLAT", "LONG"):
        raise ValueError(f"Unexpected position value: {state.position}")
    if state.position == "FLAT":
        # FLAT のときは数量=0/価格系=0を推奨
        if any(getattr(state, k) for k in ("size", "entry_price", "tp", "sl")):
            logger.warning("FLATだがエントリー関連の値が残っています（後続でクリアされます）。")
    else:
        # LONG のときは必要パラメータが正であること
        required = {"entry_price": state.entry_price, "size": state.size, "tp": state.tp, "sl": state.sl}
        for k, v in required.items():
            if v is None or float(v) <= 0:
                raise ValueError(f"LONGだが必須フィールド {k} が不正: {v}")

ensure_single_position(BotState)

## set_entry_from_order

In [38]:
def set_entry_from_order(state: BotState, order_result: Dict[str, Any]) -> BotState:
    """
    処理:
        約定結果（Step 3の戻り値）を反映して **エントリー状態** を設定する。
        単一建玉制約のため、現在が FLAT の場合のみエントリーを許可。
    INPUT:
        - state (BotState): 現在の状態
        - order_result (dict): place_market_buy() の戻り値
    OUTPUT:
        - (BotState): 更新後の状態（position="LONG", entry_price/size/tp/sl を反映）
        - 例外: 既にLONGの場合は ValueError
    """
    if state.position != "FLAT":
        raise ValueError("既にポジションを保有中のため、新規エントリーできません（単一建玉制約）。")
    state.position = "LONG"
    state.entry_price = float(order_result["price"])
    state.size = float(order_result["size"])
    state.tp = float(order_result["tp"])
    state.sl = float(order_result["sl"])
    state.entry_ts_jst = datetime.now(TZ_JST).isoformat(timespec="seconds")
    ensure_single_position(state)
    return state
# set_entry_from_order(BotState, order_result: Dict[str, Any])

## clear_to_flat()

In [39]:
def clear_to_flat(state: BotState) -> BotState:
    """
    処理:
        ポジションを解消し、FLAT に戻す（数量・価格関連をクリア）。
        PnLの計算・更新は別ステップ（損益管理）で行う想定。
    INPUT:
        - state (BotState): 現在の状態
    OUTPUT:
        - (BotState): クリア後の状態（position="FLAT", 数量/価格を0に）
    """
    state.position = "FLAT"
    state.entry_price = 0.0
    state.size = 0.0
    state.tp = 0.0
    state.sl = 0.0
    state.entry_ts_jst = None
    ensure_single_position(state)
    return state

## bump_streak()

In [40]:
def bump_streak(state: BotState, won: bool) -> BotState:
    """
    処理:
        連勝/連敗カウントを更新する。ここでは**連敗のみ**を管理（仕様に合わせ簡素化）。
    INPUT:
        - state (BotState): 現在の状態
        - won (bool): 勝ちトレードなら True、負けなら False
    OUTPUT:
        - (BotState): 更新後の状態（負けなら +1、勝ちなら 0 にリセット）
    """
    if won:
        state.streak_loss = 0
    else:
        state.streak_loss = int(state.streak_loss or 0) + 1
    return state

## touch_daily_summary_marker()

In [41]:
def touch_daily_summary_marker(state: BotState) -> BotState:
    """
    処理:
        本日の日次サマリ送信済みマーカーを更新する（23:59送信用の制御用）。
    INPUT:
        - state (BotState): 現在の状態
    OUTPUT:
        - (BotState): 更新後の状態（`last_daily_summary_date` を YYYY-MM-DD で上書き）
    """
    today = datetime.now(TZ_JST).strftime("%Y-%m-%d")
    state.last_daily_summary_date = today
    return state

## お試し

In [42]:
# ===== 使い方（例） =====
if __name__ == "__main__":
    # 1) 状態ファイルをロック付きで開く
    store = StateStore(os.path.join(STATE_DIR, "state.json"))
    try:
        with store as s:
            st = s.state
            print("Loaded state:", st)

            # 2) もしFLATで、直前のステップ（注文処理）の結果があるならエントリー反映
            mock_order = {
                "price": 100.0, "size": 50.0, "tp": 102.0, "sl": 97.0
            }
            if st.position == "FLAT":
                st = set_entry_from_order(st, mock_order)
                print("Entered LONG:", st)

            # 3) 保存（原子的置換＆バックアップ）
            s.save(st)
            print("State saved:", os.path.join(STATE_DIR, "state.json"))
    finally:
        store.release_lock()

Loaded state: BotState(position='FLAT', entry_price=0.0, size=0.0, tp=0.0, sl=0.0, pnl_cum=15921.338, streak_loss=0, last_gc_bar_ts=None, entry_ts_jst=None, last_updated_jst='2025-09-27T21:29:04+09:00', last_daily_summary_date=None)
Entered LONG: BotState(position='LONG', entry_price=100.0, size=50.0, tp=102.0, sl=97.0, pnl_cum=15921.338, streak_loss=0, last_gc_bar_ts=None, entry_ts_jst='2025-09-27T21:29:20+09:00', last_updated_jst='2025-09-27T21:29:04+09:00', last_daily_summary_date=None)
State saved: ../data/state/state.json


# 損益管理

## _ensure_metrics_log()

In [43]:
def _ensure_metrics_log():
    """
    処理:
        日次メトリクス `metrics.csv` の存在を保証（無ければヘッダを作成）。
    INPUT:
        - なし
    OUTPUT:
        - なし
    """
    path = os.path.join(METRICS_DIR, "metrics.csv")
    if not os.path.exists(path):
        cols = [
            "date","trades","win","loss","win_rate","pnl_day","pnl_cum","max_dd"
        ]
        pd.DataFrame(columns=cols).to_csv(path, index=False, encoding="utf-8")

## _append_metrics_row()

In [44]:
def _append_metrics_row(row: Dict[str, Any]):
    """
    処理:
        `metrics.csv` に1行を追記する。
    INPUT:
        - row (dict): 追記するメトリクス行（キーは `_ensure_metrics_log` の列に合わせる）
    OUTPUT:
        - なし
    """
    _ensure_metrics_log()
    path = os.path.join(METRICS_DIR, "metrics.csv")
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False, encoding="utf-8")

## _append_trade_row_close()

In [45]:
def _append_trade_row_close(row: Dict[str, Any]):
    """
    処理:
        既存の `trades.csv` に**決済行**を追記する（Step 3のヘッダに合わせてカラムを揃える）。
        buy行と同一CSVに連続記録することで、後続の集計を容易にする。
    INPUT:
        - row (dict): 追記データ（決済側）
    OUTPUT:
        - なし
    """
    path = os.path.join(TRADES_DIR, "trades.csv")
    if not os.path.exists(path):
        # Step 3の関数が未実行の環境でも壊れないよう、ヘッダを作る
        cols = [
            "ts_jst","mode","symbol","side","size","price","notional_jpy",
            "fee_jpy","slippage_bps","taker_fee_bps","tp","sl","order_id","raw"
        ]
        pd.DataFrame(columns=cols).to_csv(path, index=False, encoding="utf-8")
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False, encoding="utf-8")

## place_market_sell()

In [46]:
def place_market_sell(
    symbol: str,
    size: float,
    ref_price: float,
    params: "OrderParams",
    exchange_for_real: Optional[Any] = None
) -> Dict[str, Any]:
    """
    処理:
        **成行売り（クローズ）**を実行する。Paper は内部約定、Real は ccxt で成行売りを実行。
        手数料とスリッページを考慮し、`trades.csv` に決済行を追記する。
    INPUT:
        - symbol (str): 例 "XRP/JPY"
        - size (float): 売却数量（XRP）
        - ref_price (float): 参照価格（直近確定足の終値など）
        - params (OrderParams): mode, slippage_bps, taker_fee_bps など
        - exchange_for_real (ccxt.Exchange|None): Realモードで使う場合に渡す
    OUTPUT:
        - (dict): 約定結果（共通フィールド）
            {
              "mode","symbol","side","size","price","notional_jpy",
              "fee_jpy","order_id","raw"
            }
    """
    ts_jst = datetime.now(TZ_JST).isoformat(timespec="seconds")
    side = "sell"

    if params.mode == "paper":
        slip = params.slippage_bps / 10000.0
        fee = params.taker_fee_bps / 10000.0
        fill_price = ref_price * (1.0 - slip)  # 売りは不利側
        notional = fill_price * size
        fee_jpy = notional * fee
        order_id = f"PAPER-{int(time.time())}"
        raw = {"reason": "paper_fill_close", "ref_price": ref_price}

    elif params.mode == "real":
        from math import floor
        if exchange_for_real is None:
            exchange_for_real = _init_ccxt_for_real(params.api_key, params.secret)
        exchange_for_real.load_markets()

        adj_size = _fit_amount_to_market(exchange_for_real, symbol, size)
        if adj_size <= 0:
            raise ValueError("Calculated close size is below exchange minimum/precision.")

        order = exchange_for_real.create_order(symbol=symbol, type="market", side="sell", amount=adj_size)
        order_id = order.get("id")
        filled = float(order.get("filled", adj_size))
        avg = float(order.get("average") or order.get("price") or ref_price)
        fill_price = avg
        notional = fill_price * filled
        fee_cost = None
        if order.get("fees"):
            try:
                fee_obj = order["fees"][0]
                fee_cost = float(fee_obj.get("cost", 0.0))
            except Exception:
                fee_cost = None
        if fee_cost is None:
            fee_cost = notional * (params.taker_fee_bps / 10000.0)
        fee_jpy = fee_cost
        size = filled
        raw = order
    else:
        raise ValueError("OrderParams.mode must be 'paper' or 'real'.")

    # trades.csv へ追記
    row = {
        "ts_jst": ts_jst,
        "mode": params.mode,
        "symbol": symbol,
        "side": side,
        "size": round(size, 8),
        "price": round(fill_price, 8),
        "notional_jpy": round(notional, 2),
        "fee_jpy": round(fee_jpy, 2),
        "slippage_bps": params.slippage_bps,
        "taker_fee_bps": params.taker_fee_bps,
        "tp": None,  # 決済側は空でOK
        "sl": None,
        "order_id": order_id,
        "raw": json.dumps(raw, ensure_ascii=False),
    }
    _append_trade_row_close(row)

    return {
        "mode": params.mode,
        "symbol": symbol,
        "side": side,
        "size": float(row["size"]),
        "price": float(row["price"]),
        "notional_jpy": float(row["notional_jpy"]),
        "fee_jpy": float(row["fee_jpy"]),
        "order_id": order_id,
        "raw": raw,
    }

## is_exit_reached()

In [47]:
def is_exit_reached(current_price: float, tp: float, sl: float) -> Tuple[bool, Optional[str]]:
    """
    処理:
        現在価格が TP または SL に到達しているかを判定する（ロング前提）。
    INPUT:
        - current_price (float): 現在参照価格（直近確定足終値など）
        - tp (float): 利確水準価格
        - sl (float): 損切水準価格
    OUTPUT:
        - (Tuple[bool, str|None]): (到達しているか, 理由 "TP" or "SL" or None)
    """
    if current_price >= tp > 0:
        return True, "TP"
    if 0 < sl >= current_price:
        return True, "SL"
    return False, None

## realize_pnl_and_update_state()

In [48]:
def realize_pnl_and_update_state(
    state: "BotState",
    close_result: Dict[str, Any],
    buy_fee_jpy: float
) -> Tuple["BotState", float]:
    """
    処理:
        決済結果（売り）を受け取り、**実現損益** を計算して state を更新する。
        PnLは `(exit - entry) * size - (buy_fee + sell_fee)` とする。
        state.position は FLAT に戻し、`pnl_cum` と `streak_loss` を更新する。
    INPUT:
        - state (BotState): 決済前の状態（position="LONG" を想定）
        - close_result (dict): place_market_sell() の戻り値（'price','size','fee_jpy'等）
        - buy_fee_jpy (float): エントリー時（買い）の手数料JPY（trades.csvから拾う設計）
    OUTPUT:
        - (Tuple[BotState, float]): (更新後state, realized_pnl_jpy)
    """
    exit_price = float(close_result["price"])
    size = float(close_result["size"])
    sell_fee = float(close_result.get("fee_jpy", 0.0))
    entry_price = float(state.entry_price)

    pnl_pos = (exit_price - entry_price) * size
    pnl = pnl_pos - (buy_fee_jpy + sell_fee)

    # 累積更新
    state.pnl_cum = float(state.pnl_cum or 0.0) + pnl
    won = pnl >= 0.0
    state = bump_streak(state, won=won)

    # FLATへ
    state = clear_to_flat(state)
    return state, float(pnl)

## find_last_buy_fee_from_trades()

In [49]:
def find_last_buy_fee_from_trades(symbol: str) -> float:
    """
    処理:
        `trades.csv` から直近の **買い（side='buy'）** の行を探して手数料JPYを返す。
        決済時の費用合算のために使用。
    INPUT:
        - symbol (str): 例 "XRP/JPY"
    OUTPUT:
        - (float): 手数料JPY（見つからなければ 0.0）
    """
    path = os.path.join(TRADES_DIR, "trades.csv")
    if not os.path.exists(path):
        return 0.0
    df = pd.read_csv(path)
    df = df[(df["symbol"] == symbol) & (df["side"] == "buy")]
    if df.empty:
        return 0.0
    return float(df.iloc[-1]["fee_jpy"] or 0.0)

## append_trade_outcome_row()

In [50]:
def append_trade_outcome_row(
    symbol: str,
    reason: str,
    entry_price: float,
    exit_price: float,
    size: float,
    pnl_jpy: float
) -> None:
    """
    処理:
        トレード単位の結果行を **trades.csv** に補足追記（任意）。
        解析時の使い勝手向上のため、約定ログとは別に「集約行」を足しておく。
    INPUT:
        - symbol (str), reason (str: "TP"/"SL"), entry_price (float), exit_price (float), size (float), pnl_jpy (float)
    OUTPUT:
        - なし
    """
    ts_jst = datetime.now(TZ_JST).isoformat(timespec="seconds")
    row = {
        "ts_jst": ts_jst,
        "mode": "summary",
        "symbol": symbol,
        "side": reason,
        "size": round(size, 8),
        "price": round(exit_price, 8),
        "notional_jpy": round(exit_price * size, 2),
        "fee_jpy": round(0.0, 2),
        "slippage_bps": None,
        "taker_fee_bps": None,
        "tp": None,
        "sl": None,
        "order_id": f"SUMMARY-{int(time.time())}",
        "raw": json.dumps({"entry_price": entry_price, "exit_price": exit_price, "pnl_jpy": pnl_jpy}, ensure_ascii=False)
    }
    _append_trade_row_close(row)


## close_if_reached_and_update()

In [51]:
def close_if_reached_and_update(
    current_price: float,
    symbol: str,
    params: "OrderParams",
    store: "StateStore",
    exchange_for_real: Optional[Any] = None
) -> Optional[Dict[str, Any]]:
    """
    処理:
        現在価格（直近確定足の終値など）で **TP/SL到達を判定**し、到達していれば
        **成行売りで決済** → **実現PnL算出** → **state.json更新** → **ログ追記** を一括実行する。
    INPUT:
        - current_price (float): 判定に使う現在価格（JST確定足想定）
        - symbol (str): 例 "XRP/JPY"
        - params (OrderParams): 発注/手数料パラメータ（paper/real共通）
        - store (StateStore): `with store as s:` でロック取得済みのものを渡す
        - exchange_for_real (ccxt.Exchange|None): Realモードのときに渡すと効率的
    OUTPUT:
        - (dict|None): 決済した場合は {"reason","close_result","pnl_jpy","state"} を返す。到達していない場合は None。
        - 例外: API失敗やstate不整合は例外送出
    """
    st = store.state
    if st.position != "LONG":
        return None  # 建玉なし

    reached, reason = is_exit_reached(current_price, tp=st.tp, sl=st.sl)
    if not reached:
        return None

    # 売り約定
    close_res = place_market_sell(
        symbol=symbol,
        size=st.size,
        ref_price=current_price,
        params=params,
        exchange_for_real=exchange_for_real
    )

    # PnL算出
    buy_fee = find_last_buy_fee_from_trades(symbol)
    st_after, pnl_jpy = realize_pnl_and_update_state(st, close_res, buy_fee_jpy=buy_fee)

    # trades.csv に summary を補足
    append_trade_outcome_row(
        symbol=symbol, reason=reason,
        entry_price=st.entry_price, exit_price=close_res["price"],
        size=st.size, pnl_jpy=pnl_jpy
    )

    # state保存
    store.save(st_after)

    return {
        "reason": reason,
        "close_result": close_res,
        "pnl_jpy": pnl_jpy,
        "state": st_after,
    }

## お試し

In [52]:
if __name__ == "__main__":
    try:
        # 1) 状態ロード（ロック付き）
        state_path = "./data/state/state.json"
        st_store = StateStore(state_path)
        with st_store as s:
            st = s.state

            # 2) 仮の現在価格（通常は直近確定足の close を渡す）
            # 例: TP到達ケース（entry=100, tp=102 のときに 102.5 を渡す）
            current_price = (st.tp or 0) + 0.1

            # 3) 注文/手数料パラメータ（paper）
            op = OrderParams(mode="paper", slippage_bps=5.0, taker_fee_bps=15.0)

            # 4) 到達判定 → 決済 → PnL更新
            result = close_if_reached_and_update(
                current_price=current_price,
                symbol="XRP/JPY",
                params=op,
                store=st_store,
                exchange_for_real=None
            )
            if result is None:
                print("No close (TP/SL未到達)")
            else:
                print("Closed:", result["reason"], "PnL:", round(result["pnl_jpy"], 2))
    except Exception as e:
        logger.exception(f"PnL step error: {e}")

2025-09-27 12:29:20,756 [ERROR] PnL step error: [Errno 2] No such file or directory: './data/state/state.json.lock'
Traceback (most recent call last):
  File "/tmp/ipykernel_19633/2442555823.py", line 6, in <module>
    with st_store as s:
         ^^^^^^^^
  File "/tmp/ipykernel_19633/454459723.py", line 122, in __enter__
    ok = self.acquire_lock(timeout_sec=5.0)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_19633/454459723.py", line 32, in acquire_lock
    fd = os.open(self.lock_path, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './data/state/state.json.lock'


# 通知機能

## _http_post_json()

In [53]:
def _http_post_json(url: str, payload: Dict[str, Any], timeout: int) -> Tuple[int, str]:
    """
    処理:
        指定URLに JSON をPOST し、HTTPステータスと応答本文を返す。
    INPUT:
        - url (str): Webhook URL
        - payload (dict): 送信するJSON
        - timeout (int): タイムアウト秒
    OUTPUT:
        - (status_code, text) (Tuple[int, str]): レスポンスのHTTPステータスと本文
    """
    import requests
    resp = requests.post(url, json=payload, timeout=timeout)
    return resp.status_code, resp.text

## send_slack_message()

In [54]:
def send_slack_message(cfg: SlackConfig, text: str, blocks: Optional[List[Dict[str, Any]]] = None) -> bool:
    """
    処理:
        Slack Incoming Webhook にメッセージを送信する。未設定環境では no-op で True を返す。
        リトライを行い、失敗時は False を返す。
    INPUT:
        - cfg (SlackConfig): Webhook設定
        - text (str): プレーンテキストのメッセージ（必須）
        - blocks (list[dict]|None): Block Kit JSON（任意）
    OUTPUT:
        - (bool): 送信成功で True、失敗で False（no-op時も True）
    """
    url = cfg.resolved_url()
    if not url:
        SLACK_LOGGER.warning("SLACK_WEBHOOK_URL が未設定のため、通知はスキップします（no-op）。")
        return True  # no-opだが成功扱い

    payload = {
        "username": cfg.username,
        "icon_emoji": cfg.icon_emoji,
        "text": text
    }
    if blocks:
        payload["blocks"] = blocks

    last_err = None
    for attempt in range(1, cfg.max_retries + 1):
        try:
            status, body = _http_post_json(url, payload, timeout=cfg.timeout_sec)
            if 200 <= status < 300:
                return True
            last_err = f"HTTP {status}: {body}"
        except Exception as e:
            last_err = str(e)
        sleep_sec = (cfg.backoff_factor ** (attempt - 1))
        SLACK_LOGGER.warning(f"[{attempt}/{cfg.max_retries}] Slack送信失敗: {last_err} -> sleep {sleep_sec:.2f}s")
        time.sleep(sleep_sec)
    SLACK_LOGGER.error(f"Slack送信に失敗しました: {last_err}")
    return False

## fmt_signal_gc

In [55]:
def fmt_signal_gc(bar_ts: str, price: float, sma_s: float, sma_l: float) -> Tuple[str, List[Dict[str, Any]]]:
    """
    処理:
        ゴールデンクロス検知イベントのメッセージとblocksを生成する。
    INPUT:
        - bar_ts (str): バー時刻（JST, ISO文字列推奨）
        - price (float): 終値などの参照価格
        - sma_s (float), sma_l (float): 短期/長期SMA
    OUTPUT:
        - (text, blocks): Slack送信に使うテキストとBlock Kit配列
    """
    text = f":vertical_traffic_light: *GC detected* @ {bar_ts}  price={price:.4f}  (SMA30={sma_s:.4f} > SMA60={sma_l:.4f})"
    blocks = [
        {"type": "header", "text": {"type": "plain_text", "text": "Golden Cross Detected"}},
        {"type": "section", "text": {"type": "mrkdwn", "text": f"*Time (JST)*: `{bar_ts}`\n*Price*: `{price:.4f}`\n*SMA30/60*: `{sma_s:.4f}` / `{sma_l:.4f}`"}}
    ]
    return text, blocks

## fmt_entry()

In [56]:
def fmt_entry(symbol: str, price: float, size: float, tp: float, sl: float) -> Tuple[str, List[Dict[str, Any]]]:
    """
    処理:
        新規エントリーのメッセージとblocksを生成する。
    INPUT:
        - symbol (str), price (float), size (float), tp (float), sl (float)
    OUTPUT:
        - (text, blocks)
    """
    text = f":rocket: Entry LONG {symbol}  price={price:.4f} size={size:.4f}  TP={tp:.4f}  SL={sl:.4f}"
    blocks = [
        {"type": "header", "text": {"type": "plain_text", "text": "New Entry"}},
        {"type": "section", "fields": [
            {"type": "mrkdwn", "text": f"*Symbol:*\n{symbol}"},
            {"type": "mrkdwn", "text": f"*Price:*\n{price:.4f}"},
            {"type": "mrkdwn", "text": f"*Size:*\n{size:.4f}"},
            {"type": "mrkdwn", "text": f"*TP:*\n{tp:.4f}"},
            {"type": "mrkdwn", "text": f"*SL:*\n{sl:.4f}"},
        ]}
    ]
    return text, blocks

## fmt_close()

In [57]:
def fmt_close(reason: str, price: float, size: float, pnl_jpy: float, pnl_cum: float) -> Tuple[str, List[Dict[str, Any]]]:
    """
    処理:
        決済完了（TP/SL）のメッセージとblocksを生成する。
    INPUT:
        - reason (str): "TP" or "SL"
        - price (float): 決済価格
        - size (float): 数量
        - pnl_jpy (float): 実現損益（JPY）
        - pnl_cum (float): 累積損益（JPY）
    OUTPUT:
        - (text, blocks)
    """
    emoji = ":white_check_mark:" if pnl_jpy >= 0 else ":x:"
    text = f"{emoji} Close ({reason})  price={price:.4f} size={size:.4f}  PnL={pnl_jpy:.2f} JPY  Cum={pnl_cum:.2f}"
    blocks = [
        {"type": "header", "text": {"type": "plain_text", "text": f"Close - {reason}"}},
        {"type": "section", "fields": [
            {"type": "mrkdwn", "text": f"*Price:*\n{price:.4f}"},
            {"type": "mrkdwn", "text": f"*Size:*\n{size:.4f}"},
            {"type": "mrkdwn", "text": f"*PnL (JPY):*\n{pnl_jpy:.2f}"},
            {"type": "mrkdwn", "text": f"*PnL Cum (JPY):*\n{pnl_cum:.2f}"},
        ]}
    ]
    return text, blocks

## fmt_error()

In [58]:
def fmt_error(context: str, message: str) -> Tuple[str, List[Dict[str, Any]]]:
    """
    処理:
        エラー通知のメッセージとblocksを生成する。
    INPUT:
        - context (str): エラー発生箇所の文脈（"order", "signal", "fetch" 等）
        - message (str): エラーメッセージ
    OUTPUT:
        - (text, blocks)
    """
    text = f":warning: *Error* in `{context}` — {message}"
    blocks = [
        {"type": "section", "text": {"type": "mrkdwn", "text": f":warning: *Error* in `{context}`\n```{message}```"}},
    ]
    return text, blocks

## build_daily_summary()

In [59]:
def build_daily_summary(trades_csv_path: str, today_str: Optional[str] = None) -> Dict[str, Any]:
    """
    処理:
        trades.csv から当日の結果を集計し、日次サマリ情報を作る。
        （簡易集計：当日中の summary 行が無い場合でも約定ログから概算を作成可能）
    INPUT:
        - trades_csv_path (str): `./data/trades/trades.csv` のパス
        - today_str (str|None): "YYYY-MM-DD"。省略時はJSTの当日。
    OUTPUT:
        - (dict): サマリ情報（trades, win, loss, win_rate, pnl_day）
    """
    if not os.path.exists(trades_csv_path):
        return {"trades": 0, "win": 0, "loss": 0, "win_rate": 0.0, "pnl_day": 0.0}

    today = today_str or datetime.now(TZ_JST).strftime("%Y-%m-%d")
    df = pd.read_csv(trades_csv_path)
    if "ts_jst" not in df.columns:
        return {"trades": 0, "win": 0, "loss": 0, "win_rate": 0.0, "pnl_day": 0.0}

    # 当日の行に限定
    df["date"] = pd.to_datetime(df["ts_jst"], errors="coerce").dt.tz_localize(None).dt.strftime("%Y-%m-%d")
    dfd = df[df["date"] == today]

    pnl_day = 0.0
    trades = 0
    win = 0
    loss = 0

    # summary行があればそれを集計
    if not dfd.empty and "mode" in dfd.columns and (dfd["mode"] == "summary").any():
        sdf = dfd[dfd["mode"] == "summary"]
        for _, r in sdf.iterrows():
            try:
                raw = json.loads(r.get("raw", "{}"))
                pnl_day += float(raw.get("pnl_jpy", 0.0))
                trades += 1
                if float(raw.get("pnl_jpy", 0.0)) >= 0:
                    win += 1
                else:
                    loss += 1
            except Exception:
                continue
    else:
        # summaryが無い場合：sell行から概算（手数料はalready込みの行を想定）
        if "side" in dfd.columns:
            sells = dfd[dfd["side"] == "sell"]
            trades = len(sells)
            for _, r in sells.iterrows():
                # rawの中にentry/exit価格があれば計算、無ければnotional差分からは難しいので0扱い
                try:
                    raw = json.loads(r.get("raw", "{}"))
                    pnl = float(raw.get("pnl_jpy", 0.0))
                except Exception:
                    pnl = 0.0
                pnl_day += pnl
                if pnl >= 0:
                    win += 1
                else:
                    loss += 1

    win_rate = (win / trades * 100.0) if trades > 0 else 0.0
    return {"trades": trades, "win": win, "loss": loss, "win_rate": win_rate, "pnl_day": pnl_day}


## fmt_daily_summary()

In [60]:
def fmt_daily_summary(summary: Dict[str, Any], pnl_cum: float) -> Tuple[str, List[Dict[str, Any]]]:
    """
    処理:
        日次サマリ通知のテキストとblocksを整形する。
    INPUT:
        - summary (dict): build_daily_summary() の戻り値
        - pnl_cum (float): 累積損益（stateから参照）
    OUTPUT:
        - (text, blocks)
    """
    text = (
        f":bar_chart: Daily Summary — Trades={summary['trades']}  "
        f"Win={summary['win']}  Loss={summary['loss']}  "
        f"WinRate={summary['win_rate']:.1f}%  PnL_day={summary['pnl_day']:.2f}  PnL_cum={pnl_cum:.2f}"
    )
    fields = [
        {"type": "mrkdwn", "text": f"*Trades:*\n{summary['trades']}"},
        {"type": "mrkdwn", "text": f"*Win/Loss:*\n{summary['win']} / {summary['loss']}"},
        {"type": "mrkdwn", "text": f"*WinRate:*\n{summary['win_rate']:.1f}%"},
        {"type": "mrkdwn", "text": f"*PnL (Day):*\n{summary['pnl_day']:.2f}"},
        {"type": "mrkdwn", "text": f"*PnL (Cum):*\n{pnl_cum:.2f}"},
    ]
    blocks = [{"type": "header", "text": {"type": "plain_text", "text": "Daily Summary"}},
              {"type": "section", "fields": fields}]
    return text, blocks

## notify_gc()

In [61]:
def notify_gc(cfg: SlackConfig, bar_ts: str, price: float, sma_s: float, sma_l: float) -> bool:
    """処理: GC検知の通知を送る。INPUT: cfg, bar_ts, price, sma_s, sma_l / OUTPUT: bool(成功)"""
    text, blocks = fmt_signal_gc(bar_ts, price, sma_s, sma_l)
    return send_slack_message(cfg, text, blocks)

## notify_entry()

In [62]:
def notify_entry(cfg: SlackConfig, symbol: str, price: float, size: float, tp: float, sl: float) -> bool:
    """処理: 新規エントリー通知。INPUT: cfg, symbol, price, size, tp, sl / OUTPUT: bool"""
    text, blocks = fmt_entry(symbol, price, size, tp, sl)
    return send_slack_message(cfg, text, blocks)

## notify_close()

In [63]:
def notify_close(cfg: SlackConfig, reason: str, price: float, size: float, pnl_jpy: float, pnl_cum: float) -> bool:
    """処理: 決済通知。INPUT: cfg, reason, price, size, pnl_jpy, pnl_cum / OUTPUT: bool"""
    text, blocks = fmt_close(reason, price, size, pnl_jpy, pnl_cum)
    return send_slack_message(cfg, text, blocks)

## notify_error()

In [64]:
def notify_error(cfg: SlackConfig, context: str, message: str) -> bool:
    """処理: エラー通知。INPUT: cfg, context, message / OUTPUT: bool"""
    text, blocks = fmt_error(context, message)
    return send_slack_message(cfg, text, blocks)

## notify_daily_summary()

In [65]:
def notify_daily_summary(cfg: SlackConfig, state_path: str = "./data/state/state.json",
                         trades_csv_path: str = "./data/trades/trades.csv") -> bool:
    """
    処理:
        `state.json` と `trades.csv` を読み、当日の簡易サマリをSlackに送る。
    INPUT:
        - cfg (SlackConfig): Webhook設定
        - state_path (str): state.jsonのパス
        - trades_csv_path (str): trades.csvのパス
    OUTPUT:
        - (bool): 送信成功で True
    """
    pnl_cum = 0.0
    try:
        if os.path.exists(state_path):
            with open(state_path, "r", encoding="utf-8") as f:
                st = json.load(f)
            pnl_cum = float(st.get("pnl_cum", 0.0))
    except Exception as e:
        SLACK_LOGGER.warning(f"state.json 読込エラー: {e}")

    summary = build_daily_summary(trades_csv_path)
    text, blocks = fmt_daily_summary(summary, pnl_cum)
    return send_slack_message(cfg, text, blocks)

## お試し

In [66]:
if __name__ == "__main__":
    cfg = SlackConfig()  # 環境変数 SLACK_WEBHOOK_URL を使用
    # 1) GC検知
    notify_gc(cfg, bar_ts=datetime.now(TZ_JST).isoformat(timespec="seconds"),
              price=100.0, sma_s=99.8, sma_l=99.6)
    # 2) エントリー
    notify_entry(cfg, "XRP/JPY", price=100.0, size=50.0, tp=102.0, sl=97.0)
    # 3) 決済
    notify_close(cfg, reason="TP", price=102.3, size=50.0, pnl_jpy=110.5, pnl_cum=523.0)
    # 4) エラー通知
    notify_error(cfg, context="order", message="Insufficient funds")
    # 5) 日次サマリ
    notify_daily_summary(cfg)

# ロギング

## setup_structured_logger()

In [67]:
def setup_structured_logger(name: str = "bot") -> logging.Logger:
    """
    処理:
        アプリ全体で使う構造化ロガーを初期化する。
        - 標準ストリーム（INFO）
        - 回転ファイル `./logs/app.log`（INFO）
        - JSONLファイル `./logs/jsonl/YYYYMMDD.jsonl`（INFO）
    INPUT:
        - name (str): ロガー名
    OUTPUT:
        - (logging.Logger): 初期化済みロガー
    """
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    if logger.handlers:
        return logger  # 二重追加防止

    # コンソール
    sh = logging.StreamHandler()
    sh.setLevel(logging.INFO)
    sh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
    logger.addHandler(sh)

    # 回転ファイル（簡易: 日ごとにJSONLへも吐くため、ここは単純ファイル）
    fh = logging.FileHandler(os.path.join(LOG_DIR, "app.log"), encoding="utf-8")
    fh.setLevel(logging.INFO)
    fh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
    logger.addHandler(fh)

    # JSONL: dictを文字列化して出力するヘルパを使う（ハンドラは共通、手動でwrite）
    logger.info("Structured logger initialized.")
    return logger

## _jsonl_path_for_today()

In [68]:
def _jsonl_path_for_today() -> str:
    """処理: 当日用のJSONLパスを返す。INPUT: なし / OUTPUT: str"""
    day = datetime.now(TZ_JST).strftime("%Y%m%d")
    return os.path.join(JSONL_DIR, f"{day}.jsonl")

## write_jsonl()

In [69]:
def write_jsonl(event: Dict[str, Any]) -> None:
    """
    処理:
        構造化イベントを1行JSONで `./logs/jsonl/YYYYMMDD.jsonl` に追記する。
    INPUT:
        - event (dict): シリアライズ可能な辞書（時刻は関数側で付与）
    OUTPUT:
        - なし（失敗は例外）
    """
    event = dict(event or {})
    event.setdefault("ts_jst", datetime.now(TZ_JST).isoformat(timespec="seconds"))
    path = _jsonl_path_for_today()
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(event, ensure_ascii=False) + "\n")

## log_api_call()

In [70]:
def log_api_call(logger: logging.Logger, name: str, request: Dict[str, Any], response: Dict[str, Any],
                 ok: bool, latency_ms: float) -> None:
    """
    処理:
        API呼び出しの結果を構造化して記録する（コンソール/ファイル/JSONL）。
    INPUT:
        - logger (logging.Logger): 事前に setup_structured_logger() したもの
        - name (str): API名（例: "ccxt.fetch_ohlcv"）
        - request (dict): 入力パラメータの要約（秘匿情報は含めない）
        - response (dict): 返却値の要約（成功/失敗で適宜）
        - ok (bool): 成否
        - latency_ms (float): レイテンシ（ms）
    OUTPUT:
        - なし
    """
    level = logging.INFO if ok else logging.WARNING
    logger.log(level, f"{name} ok={ok} latency_ms={latency_ms:.1f}")
    write_jsonl({
        "type": "api_call",
        "name": name,
        "ok": ok,
        "latency_ms": latency_ms,
        "request": request,
        "response": response,
    })

## log_exception()

In [71]:
def log_exception(logger: logging.Logger, context: str, exc: Exception) -> None:
    """
    処理:
        例外情報をトレース付きで構造化ログに記録する。
    INPUT:
        - logger (logging.Logger): ロガー
        - context (str): 例外の発生箇所識別子（"order", "signal", "fetch" など）
        - exc (Exception): 例外オブジェクト
    OUTPUT:
        - なし
    """
    trace = "".join(traceback.format_exception(type(exc), exc, exc.__traceback__))
    logger.error(f"Exception in {context}: {exc}")
    write_jsonl({
        "type": "exception",
        "context": context,
        "error": str(exc),
        "traceback": trace,
    })

## append_trade_log()

In [72]:
def append_trade_log(side: str, price: float, size: float, mode: str = "paper",
                     symbol: str = "XRP/JPY", fee_jpy: float = 0.0,
                     slippage_bps: Optional[float] = None, taker_fee_bps: Optional[float] = None,
                     order_id: Optional[str] = None, extra: Optional[Dict[str, Any]] = None) -> None:
    """
    処理:
        Step 3/5 と同じレイアウトで `trades.csv` に1行を追記するヘルパ。
        既存の実装にフック/置換しやすいように提供。
    INPUT:
        - side (str): "buy" | "sell" | "summary" など
        - price (float), size (float), mode (str), symbol (str), fee_jpy (float)
        - slippage_bps (float|None), taker_fee_bps (float|None)
        - order_id (str|None): 任意の識別子
        - extra (dict|None): rawカラムに入れる任意情報
    OUTPUT:
        - なし（失敗は例外）
    """
    path = os.path.join(TRADES_DIR, "trades.csv")
    if not os.path.exists(path):
        cols = [
            "ts_jst","mode","symbol","side","size","price","notional_jpy",
            "fee_jpy","slippage_bps","taker_fee_bps","tp","sl","order_id","raw"
        ]
        pd.DataFrame(columns=cols).to_csv(path, index=False, encoding="utf-8")

    row = {
        "ts_jst": datetime.now(TZ_JST).isoformat(timespec="seconds"),
        "mode": mode,
        "symbol": symbol,
        "side": side,
        "size": round(float(size), 8),
        "price": round(float(price), 8),
        "notional_jpy": round(float(price) * float(size), 2),
        "fee_jpy": round(float(fee_jpy), 2),
        "slippage_bps": slippage_bps,
        "taker_fee_bps": taker_fee_bps,
        "tp": None,
        "sl": None,
        "order_id": order_id or f"LOG-{int(time.time())}",
        "raw": json.dumps(extra or {}, ensure_ascii=False),
    }
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False, encoding="utf-8")
    write_jsonl({"type": "trade_log", **row})

## _ensure_metrics_log()

In [73]:
def _ensure_metrics_log():
    """処理: `metrics.csv` を存在させる（列: date,trades,win,loss,win_rate,pnl_day,pnl_cum,max_dd）"""
    path = os.path.join(METRICS_DIR, "metrics.csv")
    if not os.path.exists(path):
        cols = ["date","trades","win","loss","win_rate","pnl_day","pnl_cum","max_dd"]
        pd.DataFrame(columns=cols).to_csv(path, index=False, encoding="utf-8")

## _equity_curve_from_trades()

In [74]:
def _equity_curve_from_trades(trades_csv_path: str) -> pd.Series:
    """
    処理:
        trades.csv の *summary 行*（Step 5で追記）を読み、日中の累積損益カーブ（equity）を構築する。
        summaryが無い場合は空Seriesを返す。
    INPUT:
        - trades_csv_path (str): 取引ログのパス
    OUTPUT:
        - (pd.Series): index=datetime(JST naive), values=cumulative PnL（当日内の増分累積）
    """
    if not os.path.exists(trades_csv_path):
        return pd.Series(dtype=float)
    df = pd.read_csv(trades_csv_path)
    if df.empty or "mode" not in df.columns:
        return pd.Series(dtype=float)
    sdf = df[df["mode"] == "summary"].copy()
    if sdf.empty:
        return pd.Series(dtype=float)
    # 末尾の raw に pnl_jpy が入っている想定
    pnl_list = []
    ts_list = []
    for _, r in sdf.iterrows():
        ts = r.get("ts_jst")
        try:
            raw = json.loads(r.get("raw", "{}"))
            pnl = float(raw.get("pnl_jpy", 0.0))
        except Exception:
            pnl = 0.0
        pnl_list.append(pnl)
        try:
            ts_list.append(pd.to_datetime(ts))
        except Exception:
            ts_list.append(pd.NaT)
    eq = pd.Series(pnl_list, index=pd.to_datetime(ts_list)).sort_index()
    eq = eq.cumsum()
    return eq


## _max_drawdown_from_equity()

In [75]:
def _max_drawdown_from_equity(equity: pd.Series) -> float:
    """
    処理:
        累積損益カーブから **最大ドローダウン** を計算する。
        DD定義: 過去最大値からの下落幅（JPY）。
    INPUT:
        - equity (pd.Series): 時系列の累積PnL（index: datetime, values: float）
    OUTPUT:
        - (float): 最大ドローダウン（負の値を返さず、下落幅の絶対値[JPY]で返す）
    """
    if equity.empty:
        return 0.0
    rolling_max = equity.cummax()
    dd = equity - rolling_max
    max_dd = dd.min()  # 最小（最大下落）
    return float(abs(max_dd))

## write_daily_metrics()

In [76]:
def write_daily_metrics(trades_csv_path: str, state_path: str) -> Dict[str, Any]:
    """
    処理:
        当日のトレード結果から **日次メトリクス** を集計し、`metrics.csv` に追記する。
        - trades / win / loss / win_rate / pnl_day / pnl_cum / **max_dd**
        - pnl_cum は state.json の値を参照
    INPUT:
        - trades_csv_path (str): `./data/trades/trades.csv`
        - state_path (str): `./data/state/state.json`
    OUTPUT:
        - (dict): 追記したメトリクス行（返却用）
    """
    # 当日判定
    today = datetime.now(TZ_JST).strftime("%Y-%m-%d")
    # 当日の summary 行から集計
    trades = 0
    win = 0
    loss = 0
    pnl_day = 0.0

    if os.path.exists(trades_csv_path):
        df = pd.read_csv(trades_csv_path)
        if not df.empty and "ts_jst" in df.columns:
            df["date"] = pd.to_datetime(df["ts_jst"], errors="coerce").dt.tz_localize(None).dt.strftime("%Y-%m-%d")
            dfd = df[df["date"] == today]
            sdf = dfd[dfd.get("mode", "") == "summary"]
            for _, r in sdf.iterrows():
                try:
                    raw = json.loads(r.get("raw", "{}"))
                    pnl = float(raw.get("pnl_jpy", 0.0))
                except Exception:
                    pnl = 0.0
                pnl_day += pnl
                trades += 1
                if pnl >= 0:
                    win += 1
                else:
                    loss += 1

    win_rate = (win / trades * 100.0) if trades > 0 else 0.0

    # 累積PnLは state.json を参照
    pnl_cum = 0.0
    try:
        with open(state_path, "r", encoding="utf-8") as f:
            st = json.load(f)
        pnl_cum = float(st.get("pnl_cum", 0.0))
    except Exception:
        pass

    # 当日内の最大ドローダウン（summaryの累積カーブから推定）
    eq = _equity_curve_from_trades(trades_csv_path)
    # 当日分に限定（indexがNaTの可能性対策）
    try:
        eqd = eq[eq.index.tz_localize(None).strftime("%Y-%m-%d") == today]
    except Exception:
        eqd = pd.Series(dtype=float)
    max_dd = _max_drawdown_from_equity(eqd)

    # 追記
    _ensure_metrics_log()
    metrics_path = os.path.join(METRICS_DIR, "metrics.csv")
    mrow = {
        "date": today,
        "trades": trades,
        "win": win,
        "loss": loss,
        "win_rate": round(win_rate, 2),
        "pnl_day": round(pnl_day, 2),
        "pnl_cum": round(pnl_cum, 2),
        "max_dd": round(max_dd, 2),
    }
    dfm = pd.read_csv(metrics_path)
    # 既に当日行がある場合は置き換え（冪等）
    dfm = dfm[dfm["date"] != today]
    dfm = pd.concat([dfm, pd.DataFrame([mrow])], ignore_index=True)
    dfm.to_csv(metrics_path, index=False, encoding="utf-8")

    # JSONLにも書いておく
    write_jsonl({"type": "daily_metrics", **mrow})
    return mrow


## お試し

In [77]:
if __name__ == "__main__":
    logger = setup_structured_logger("bot")

    # APIログ例（疑似）
    t0 = time.time()
    try:
        # ここでAPI呼び出しを行った想定
        time.sleep(0.05)
        resp = {"rows": 200}
        ok = True
    except Exception as e:
        ok = False
        resp = {"error": str(e)}
        log_exception(logger, "api_example", e)
    finally:
        latency = (time.time() - t0) * 1000.0
        log_api_call(logger, "example_api", {"param": 1}, resp, ok, latency)

    # トレードログ例
    append_trade_log(side="summary", price=102.0, size=50.0, mode="summary",
                     fee_jpy=0.0, order_id="SUMMARY-EXAMPLE",
                     extra={"entry_price": 100.0, "exit_price": 102.0, "pnl_jpy": 100.0})

    # 日次メトリクス書き出し
    m = write_daily_metrics(os.path.join(TRADES_DIR, "trades.csv"),
                            os.path.join(STATE_DIR, "state.json"))
    print("Daily metrics:", m)

2025-09-27 12:29:21,949 [INFO] Structured logger initialized.
2025-09-27 12:29:22,000 [INFO] example_api ok=True latency_ms=50.1


Daily metrics: {'date': '2025-09-27', 'trades': 5, 'win': 5, 'loss': 0, 'win_rate': 100.0, 'pnl_day': 16315.39, 'pnl_cum': 15921.34, 'max_dd': 0.0}


/tmp/ipykernel_19633/2133922204.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


# 統合RUN

## _check_dependencies()

In [78]:
# --- 既存実装からの参照（未定義なら異常にする） ---
required_symbols = [
    "CCXTConfig","fetch_ohlcv_latest_ccxt",
    "SignalParams","add_sma_columns","detect_golden_cross_latest","update_state_after_signal",
    "OrderParams","place_market_buy","should_open_from_signal","update_state_on_entry",
    "BotState","StateStore","set_entry_from_order",
    "close_if_reached_and_update",
    "SlackConfig","notify_gc","notify_entry","notify_close","notify_error",
    "setup_structured_logger","log_api_call","log_exception","write_jsonl",
]

def _check_dependencies():
    missing = [s for s in required_symbols if s not in globals()]
    if missing:
        raise RuntimeError(f"以下のシンボルが未定義です。先に各ステップのセルを実行してください: {missing}")


## _env_or()

In [79]:
def _env_or(default: Optional[str], *keys: str) -> Optional[str]:
    """処理: 環境変数のいずれかから値を取得（最初に見つかったもの）。INPUT: default, *keys / OUTPUT: str|None"""
    for k in keys:
        v = os.getenv(k)
        if v:
            return v
    return default

## RunnerConfig

In [80]:
@dataclass
class RunnerConfig:
    """
    処理:
        統合ランナー用の設定パラメータを保持する。
    INPUT:
        - mode (str): "paper" | "real"
        - symbol (str): 取引ペア（例: "XRP/JPY"）
        - state_path (str): state.json の保存パス
        - notional_jpy (float): 1回あたりの想定発注額（JPY）
        - slippage_bps (float): スリッページ（bps）
        - taker_fee_bps (float): 手数料（bps）
        - api_key (str|None), secret (str|None): realモード時に使用
    OUTPUT:
        - なし（設定値の保持のみ）
    """
    mode: str = "paper"
    symbol: str = "XRP/JPY"
    state_path: str = "../data/state/state.json"
    notional_jpy: float = 5000.0
    slippage_bps: float = 5.0
    taker_fee_bps: float = 15.0
    api_key: Optional[str] = None
    secret: Optional[str] = None

## run_hourly_cycle()

In [81]:
def run_hourly_cycle(cfg: RunnerConfig) -> Dict[str, Any]:
    """
    処理:
        1時間サイクルの**単発実行**を行う統合関数。
        - データ取得（直近200本の1h確定足）
        - SMA30/60計算 → 直近確定足でのGC判定（重複抑止）
        - FLATかつGCなら成行買い（Paper/Real）→ state保存 → 通知
        - LONGならTP/SL到達判定 → 成行売り → PnL反映 → state保存 → 通知
        - 各所でログ（API/例外/イベント）を記録
    INPUT:
        - cfg (RunnerConfig): ランナー設定
    OUTPUT:
        - (dict): 実行サマリ（key例: stage, signal, order, close, state_meta など）
    """
    _check_dependencies()
    logger = setup_structured_logger("runner")
    slack = SlackConfig()  # SLACK_WEBHOOK_URL は環境変数から

    summary: Dict[str, Any] = {"stage": None}

    # --- State をロック付きで開く ---
    store = StateStore(cfg.state_path)
    try:
        with store as s:
            st = s.state  # BotState

            # --- 1) データ取得（ccxt） ---
            t0 = time.time()
            ccxt_cfg = CCXTConfig(limit=200, timeframe="1h", period_sec=3600)
            try:
                df = fetch_ohlcv_latest_ccxt(ccxt_cfg)
                ok = True
                resp_summary = {"rows": len(df)}
            except Exception as e:
                ok = False
                resp_summary = {"error": str(e)}
                log_exception(logger, "fetch_ohlcv_latest_ccxt", e)
                notify_error(slack, "fetch", str(e))
                raise
            finally:
                latency_ms = (time.time() - t0) * 1000.0
                log_api_call(logger, "ccxt.fetch_ohlcv_or_trades", {"symbol": ccxt_cfg.symbol, "timeframe": ccxt_cfg.timeframe}, resp_summary, ok, latency_ms)

            summary["stage"] = "fetched"
            write_jsonl({"type": "stage", "name": "fetched", "rows": len(df)})

            # --- 2) シグナル（SMA30/60 & 直近確定足GC） ---
            sig_params = SignalParams()
            df_feat = add_sma_columns(df, sig_params)

            last_gc_ts = st.last_gc_bar_ts
            signal = detect_golden_cross_latest(df_feat, sig_params, last_signaled_bar_ts=last_gc_ts)
            summary["signal"] = {
                "is_gc": signal["is_gc"],
                "already": signal["already_signaled"],
                "bar_ts": str(signal["bar_ts"]),
                "price": signal["price"]
            }
            write_jsonl({"type": "signal", **summary["signal"]})

            # 通知（GC検知のみ）
            if signal["is_gc"] and not signal["already_signaled"]:
                try:
                    notify_gc(
                        slack,
                        bar_ts=str(signal["bar_ts"]),
                        price=signal["price"],
                        sma_s=signal["sma_short"],
                        sma_l=signal["sma_long"],
                    )
                except Exception as e:
                    log_exception(logger, "notify_gc", e)

            # --- 3) エントリー or 4) 決済判定 ---
            order_res = None
            close_res = None

            if should_open_from_signal(asdict(st), signal):
                # 3) エントリー
                op = OrderParams(
                    mode=cfg.mode,
                    notional_jpy=cfg.notional_jpy,
                    slippage_bps=cfg.slippage_bps,
                    taker_fee_bps=cfg.taker_fee_bps,
                    api_key=_env_or(cfg.api_key, "BFX_API_KEY", "BITFLYER_API_KEY"),
                    secret=_env_or(cfg.secret, "BFX_API_SECRET", "BITFLYER_API_SECRET"),
                )
                try:
                    order_res = place_market_buy(cfg.symbol, signal["price"], op)
                    st = set_entry_from_order(st, order_res)
                    s.save(st)  # 保存
                    summary["order"] = {k: order_res[k] for k in ["mode","price","size","tp","sl","order_id"]}
                    write_jsonl({"type": "entry", **summary["order"]})
                    try:
                        notify_entry(slack, cfg.symbol, order_res["price"], order_res["size"], order_res["tp"], order_res["sl"])
                    except Exception as e:
                        log_exception(logger, "notify_entry", e)
                except Exception as e:
                    log_exception(logger, "place_market_buy", e)
                    notify_error(slack, "order_entry", str(e))
                    raise
            else:
                # 4) 決済判定（LONG 中のみ）
                current_close = float(df_feat.iloc[-1]["close"])
                op = OrderParams(
                    mode=cfg.mode,
                    notional_jpy=cfg.notional_jpy,
                    slippage_bps=cfg.slippage_bps,
                    taker_fee_bps=cfg.taker_fee_bps,
                    api_key=_env_or(cfg.api_key, "BFX_API_KEY", "BITFLYER_API_KEY"),
                    secret=_env_or(cfg.secret, "BFX_API_SECRET", "BITFLYER_API_SECRET"),
                )
                try:
                    res = close_if_reached_and_update(current_price=current_close, symbol=cfg.symbol, params=op, store=s)
                    if res is not None:
                        close_res = res
                        summary["close"] = {
                            "reason": res["reason"],
                            "price": res["close_result"]["price"],
                            "size": res["close_result"]["size"],
                            "pnl_jpy": res["pnl_jpy"],
                            "pnl_cum": res["state"].pnl_cum,
                        }
                        write_jsonl({"type": "close", **summary["close"]})
                        try:
                            notify_close(slack, res["reason"], res["close_result"]["price"], res["close_result"]["size"], res["pnl_jpy"], res["state"].pnl_cum)
                        except Exception as e:
                            log_exception(logger, "notify_close", e)
                except Exception as e:
                    log_exception(logger, "close_if_reached_and_update", e)
                    notify_error(slack, "order_close", str(e))
                    raise

            # --- 5) GC多重検知抑止のため state を更新 ---
            st_dict = asdict(st)
            st_new = update_state_after_signal(st_dict, signal)
            st.last_gc_bar_ts = st_new.get("last_gc_bar_ts", st.last_gc_bar_ts)
            s.save(st)

            summary["state_meta"] = {"position": st.position, "pnl_cum": st.pnl_cum, "last_gc_bar_ts": st.last_gc_bar_ts}
            write_jsonl({"type": "stage", "name": "done", **summary["state_meta"]})
            return summary

    except Exception as e:
        try:
            notify_error(SlackConfig(), "runner", str(e))
        except Exception:
            pass
        raise

## paper実行

### 単発実行

In [82]:
cfg = RunnerConfig(
    mode="paper",          # 実運用は "real"
    notional_jpy=5000.0,
    slippage_bps=5.0,
    taker_fee_bps=15.0,
)
out = run_hourly_cycle(cfg)
out

2025-09-27 12:29:22,073 [INFO] Structured logger initialized.
2025-09-27 12:29:23,305 [INFO] ccxt.fetch_ohlcv_or_trades ok=True latency_ms=1230.1
/tmp/ipykernel_19633/2360316234.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
/tmp/ipykernel_19633/2360316234.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


{'stage': 'fetched',
 'signal': {'is_gc': False,
  'already': False,
  'bar_ts': '2025-09-27 20:00:00+09:00',
  'price': 416.3},
 'close': {'reason': 'TP',
  'price': 416.09185,
  'size': 50.0,
  'pnl_jpy': 15772.632500000002,
  'pnl_cum': 31693.970500000003},
 'state_meta': {'position': 'FLAT',
  'pnl_cum': 31693.970500000003,
  'last_gc_bar_ts': None}}

### 一時間おきに実行

In [ ]:
import time
from datetime import datetime

### config
cfg = RunnerConfig(mode="paper")  # or "real"
print(cfg)

### 
while True:
    now = datetime.now()
    print(f"=== {now:%Y-%m-%d %H:%M:%S} start ===")
    try:
        out = run_hourly_cycle(cfg)
        print("summary:", out)
    except Exception as e:
        print("error:", e)

    # 次の実行タイミングを計算（毎分01秒）
    now = datetime.now()
    next_run = (now.replace(second=1, microsecond=0) 
                + timedelta(minutes=1) if now.second >= 1 else now.replace(second=1, microsecond=0))
    sleep_sec = (next_run - datetime.now()).total_seconds()
    print(f"sleep {sleep_sec:.1f} sec until {next_run}")
    time.sleep(max(0, sleep_sec))

RunnerConfig(mode='paper', symbol='XRP/JPY', state_path='../data/state/state.json', notional_jpy=5000.0, slippage_bps=5.0, taker_fee_bps=15.0, api_key=None, secret=None)
=== 2025-09-27 12:29:23 start ===


2025-09-27 12:29:24,746 [INFO] ccxt.fetch_ohlcv_or_trades ok=True latency_ms=1199.7


summary: {'stage': 'fetched', 'signal': {'is_gc': False, 'already': False, 'bar_ts': '2025-09-27 20:00:00+09:00', 'price': 416.3}, 'state_meta': {'position': 'FLAT', 'pnl_cum': 31693.970500000003, 'last_gc_bar_ts': None}}
sleep 36.2 sec until 2025-09-27 12:30:01
=== 2025-09-27 12:30:01 start ===


2025-09-27 12:30:05,226 [INFO] ccxt.fetch_ohlcv_or_trades ok=True latency_ms=4225.6


summary: {'stage': 'fetched', 'signal': {'is_gc': False, 'already': False, 'bar_ts': '2025-09-27 20:00:00+09:00', 'price': 416.3}, 'state_meta': {'position': 'FLAT', 'pnl_cum': 31693.970500000003, 'last_gc_bar_ts': None}}
sleep 55.8 sec until 2025-09-27 12:31:01
=== 2025-09-27 12:31:01 start ===


...

summary: {'stage': 'fetched', 'signal': {'is_gc': False, 'already': False, 'bar_ts': '2025-09-27 23:00:00+09:00', 'price': 417.12}, 'state_meta': {'position': 'FLAT', 'pnl_cum': 31693.970500000003, 'last_gc_bar_ts': None}}
sleep 56.7 sec until 2025-09-27 15:49:01
=== 2025-09-27 16:22:01 start ===


2025-09-27 16:22:03,542 [INFO] ccxt.fetch_ohlcv_or_trades ok=True latency_ms=2541.3


summary: {'stage': 'fetched', 'signal': {'is_gc': False, 'already': False, 'bar_ts': '2025-09-28 00:00:00+09:00', 'price': 417.17}, 'state_meta': {'position': 'FLAT', 'pnl_cum': 31693.970500000003, 'last_gc_bar_ts': None}}
sleep 57.5 sec until 2025-09-27 16:23:01
=== 2025-09-27 16:49:01 start ===


2025-09-27 16:49:03,194 [INFO] ccxt.fetch_ohlcv_or_trades ok=True latency_ms=2193.0


summary: {'stage': 'fetched', 'signal': {'is_gc': False, 'already': False, 'bar_ts': '2025-09-28 00:00:00+09:00', 'price': 417.17}, 'state_meta': {'position': 'FLAT', 'pnl_cum': 31693.970500000003, 'last_gc_bar_ts': None}}
sleep 57.8 sec until 2025-09-27 16:50:01
